# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-02f3f9b172-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Sport Workflow Series ("World Records explorative search") 

Consider the following exploratory information need:

> compile a list of athletes that held world records across some disciplines

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q1189`    | Usain Bolt    | node      |
| `wd:Q766`     | Jamaica       | node |
| `wd:Q688615`  | World Record  | node |
| `wd:Q542`     | athletics     | node |



Also consider

```
?a wdt:P106/wdt:P279 wd:Q2066131
```

is the BGP to retrieve all instances of **athlete**

```
?a wdt:P279 wd:Q688615
```

is the BGP to retrieve the types of **world records**


## Workload Goals

1. Identify the BGP for detaining a world record

2. Identify the BGP for the types of competitions in which a world record is classified

3. How many world records are held by Italian athletes

4. Compare number of world records held across different dimensions
 
   4.1 In which specific sport France has the most world records
   
   4.2 How many world records are held across genders
   
   4.3 Which sport has the highest number of world records


In [3]:
# start your workflow here

In [4]:
queryString = """
SELECT COUNT(*)
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '929008')]


1

In [5]:
#understand the node wd:Q2066131

queryString = """
SELECT ?name
WHERE { 

 wd:Q2066131 <http://schema.org/name> ?name.

} 
"""

print("Results")
run_query(queryString)

Results
[('name', 'athlete')]


1

In [6]:
#return the node connected to Athlete (node) through wdt:P279 (subclass)

queryString = """
SELECT ?obj ?name
WHERE { 

     wd:Q2066131 wdt:P279 ?obj.
     ?obj <http://schema.org/name> ?name.

} 
"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q50995749'), ('name', 'sportsperson')]


1

In [6]:
#return athletes

queryString = """
SELECT ?ath ?aName
WHERE { 

     ?ath wdt:P106 wd:Q50995749.
     ?ath <http://schema.org/name> ?aName.

} 
"""

print("Results")
run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q11861652'), ('name', 'Harry Lindblad')]
[('sub', 'http://www.wikidata.org/entity/Q4886265'), ('name', 'Ben Pakulski')]
[('sub', 'http://www.wikidata.org/entity/Q61110529'), ('name', 'Laive Poska')]
[('sub', 'http://www.wikidata.org/entity/Q12377124'), ('name', 'Toomas Sildmäe')]
[('sub', 'http://www.wikidata.org/entity/Q66122090'), ('name', 'Teresa McDonnell')]
[('sub', 'http://www.wikidata.org/entity/Q70647115'), ('name', 'Oluwaseun Oyegunle')]
[('sub', 'http://www.wikidata.org/entity/Q2095099'), ('name', 'Alex Shibutani')]
[('sub', 'http://www.wikidata.org/entity/Q69418107'), ('name', 'Kyle Alexander')]
[('sub', 'http://www.wikidata.org/entity/Q61104393'), ('name', 'Aarne K. Leskinen')]
[('sub', 'http://www.wikidata.org/entity/Q61504572'), ('name', 'Allan Coulter')]
[('sub', 'http://www.wikidata.org/entity/Q66091792'), ('name', 'Barry Brooker')]
[('sub', 'http://www.wikidata.org/entity/Q60684939'), ('name', 'Jack Devine')]
[('sub', 'h

134

In [3]:
#outgoing edges of Athlete node

queryString = """
SELECT DISTINCT ?pred ?pName ?obj ?objName
WHERE { 
    wd:Q2066131 ?pred ?obj.
    ?pred <http://schema.org/name> ?pName.
    ?obj  <http://schema.org/name> ?objName.
} 
ORDER BY ?pName
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P6186'), ('pName', 'category for eponymous categories'), ('obj', 'http://www.wikidata.org/entity/Q8923229'), ('objName', 'Category:Wikipedia categories named after sportspeople')]
[('pred', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q106727050'), ('objName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('pred', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q11513337'), ('objName', 'athletics competitor')]
[('pred', 'http://www.wikidata.org/prop/direct/P425'), ('pName', 'field of this occupation'), ('obj', 'http://www.wikidata.org/entity/Q349'), ('objName', 'sport')]
[('pred', 'http://www.wikidata.org/prop/direct/P425'), ('pName', 'field of this occupation'), ('obj', 'http://www.wikidata.org/entity/Q920057'), ('objName', "women's sports")]
[('pred', 'http://www.wikidata.org/prop/dire

13

In [11]:
#obj not literal

queryString = """
SELECT ?prep ?pName ?obj ?oName
WHERE { 
    wd:Q2066131 ?prep ?obj.
    FILTER(!isLiteral(?obj))
    ?prep <http://schema.org/name> ?pName.
    ?obj <http://schema.org/name> ?oName.
} 
"""

print("Results")
run_query(queryString)

Results
[('prep', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q106727050'), ('oName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('prep', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q11513337'), ('oName', 'athletics competitor')]
[('prep', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q12737077'), ('oName', 'occupation')]
[('prep', 'http://www.wikidata.org/prop/direct/P417'), ('pName', 'patron saint'), ('obj', 'http://www.wikidata.org/entity/Q183332'), ('oName', 'Saint Sebastian')]
[('prep', 'http://www.wikidata.org/prop/direct/P425'), ('pName', 'field of this occupation'), ('obj', 'http://www.wikidata.org/entity/Q349'), ('oName', 'sport')]
[('prep', 'http://www.wikidata.org/prop/direct/P425'), ('pName', 'field of this occupation'), ('obj', 'http://www.wikidata.org/entit

13

In [19]:
#incoming edges of Athlete node

queryString = """
SELECT DISTINCT ?pName ?sName
WHERE { 
    ?sub ?pred wd:Q2066131.
    ?pred <http://schema.org/name> ?pName.
    ?sub <http://schema.org/name> ?sName.
} 
ORDER BY ?pName
"""

print("Results")
run_query(queryString)

Results
[('pName', 'afflicts'), ('sName', 'sudden death in athletes')]
[('pName', 'afflicts'), ('sName', 'Post-traumatic stress disorder among athletes')]
[('pName', 'category combines topics'), ('sName', 'Category:Russian sportsmen')]
[('pName', 'category combines topics'), ('sName', 'Category:Olympic medalists for Spain')]
[('pName', 'category combines topics'), ('sName', 'Category:French sportspeople')]
[('pName', 'category combines topics'), ('sName', 'Category:Zimbabwean sportsmen')]
[('pName', 'category combines topics'), ('sName', 'Category:Competitors at the 2015 European Games')]
[('pName', 'category combines topics'), ('sName', 'Category:Sportspeople from Pernambuco')]
[('pName', 'category combines topics'), ('sName', 'Category:Sportspeople from Roraima')]
[('pName', 'category combines topics'), ('sName', 'Category:Chinese sportspeople')]
[('pName', 'category combines topics'), ('sName', 'Category:American sportspeople')]
[('pName', 'category combines topics'), ('sName', 'Cat

22279

In [25]:
#subject of Athlete connected through wdt:P106 (profession)

queryString = """
SELECT DISTINCT ?sub ?sName
WHERE { 
    ?sub wdt:P106 wd:Q2066131.
    ?sub <http://schema.org/name> ?sName.
} 

ORDER BY DESC (?sName)
LIMIT 500
"""

print("Results")
run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q12219643'), ('sName', 'Ṣāliḥ Jāsim Shihāb')]
[('sub', 'http://www.wikidata.org/entity/Q66305529'), ('sName', 'ציון חצרוני')]
[('sub', 'http://www.wikidata.org/entity/Q107395392'), ('sName', 'Գևորգ Ալիխանյան')]
[('sub', 'http://www.wikidata.org/entity/Q47809460'), ('sName', 'Ștefania Botcariu')]
[('sub', 'http://www.wikidata.org/entity/Q21857485'), ('sName', 'Ștefan Kaban')]
[('sub', 'http://www.wikidata.org/entity/Q65029762'), ('sName', 'Ștefan Ionescu')]
[('sub', 'http://www.wikidata.org/entity/Q4025787'), ('sName', 'Ștefan Ardeleanu')]
[('sub', 'http://www.wikidata.org/entity/Q43670283'), ('sName', 'Živa Čopi')]
[('sub', 'http://www.wikidata.org/entity/Q30123005'), ('sName', 'Željko Petrović')]
[('sub', 'http://www.wikidata.org/entity/Q28016786'), ('sName', 'Željko Milanović')]
[('sub', 'http://www.wikidata.org/entity/Q28363992'), ('sName', 'Željko Kosanović')]
[('sub', 'http://www.wikidata.org/entity/Q10280286'), ('sName', 'Železný Z

500

In [8]:
#check some relations with wd:Q452 (outgoing edges)

queryString = """
SELECT DISTINCT ?sub ?sName ?pred ?pName
WHERE { 
    ?sub wdt:P279 wd:Q2066131.
    ?sub ?pred wd:Q542.
    ?sub <http://schema.org/name> ?sName.
    ?pred <http://schema.org/name> ?pName.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q11513337'), ('sName', 'athletics competitor'), ('pred', 'http://www.wikidata.org/prop/direct/P425'), ('pName', 'field of this occupation')]


1

In [29]:
#return the disciplines 

queryString = """
SELECT DISTINCT ?sub ?sName 
WHERE { 
    ?sub wdt:P279 wd:Q2066131.
    ?sub wdt:P425 ?obj.
    ?sub <http://schema.org/name> ?sName.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q65924683'), ('sName', 'wushu practitioner')]
[('sub', 'http://www.wikidata.org/entity/Q16278103'), ('sName', 'squash player')]
[('sub', 'http://www.wikidata.org/entity/Q18667447'), ('sName', 'pesäpallo player')]
[('sub', 'http://www.wikidata.org/entity/Q29579227'), ('sName', 'bowls player')]
[('sub', 'http://www.wikidata.org/entity/Q61278580'), ('sName', 'roller skater')]
[('sub', 'http://www.wikidata.org/entity/Q78063080'), ('sName', 'rinkball player')]
[('sub', 'http://www.wikidata.org/entity/Q23754399'), ('sName', 'korfball player')]
[('sub', 'http://www.wikidata.org/entity/Q17611899'), ('sName', 'racquetball player')]
[('sub', 'http://www.wikidata.org/entity/Q13382566'), ('sName', 'canoeist')]
[('sub', 'http://www.wikidata.org/entity/Q13581129'), ('sName', 'orienteer')]
[('sub', 'http://www.wikidata.org/entity/Q16004471'), ('sName', 'kayaker')]
[('sub', 'http://www.wikidata.org/entity/Q17318006'), ('sName', 'freediver')]
[('sub', 'h

100

In [4]:
#check some relations with wd:Q452 (outcoming edges)

queryString = """
SELECT DISTINCT  ?prep ?pName ?sub ?sName
WHERE { 
    ?sub wdt:P279 wd:Q2066131.
    wd:Q542 ?prep ?sub .
    FILTER(!isLiteral(?sub))
    ?sub <http://schema.org/name> ?sName.
    ?prep <http://schema.org/name> ?pName.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('prep', 'http://www.wikidata.org/prop/direct/P3095'), ('pName', 'practiced by'), ('sub', 'http://www.wikidata.org/entity/Q11513337'), ('sName', 'athletics competitor')]


1

In [30]:
#check if there exists relations with Usain Bolt (in the initial query)
#Usain - prep - a - ....

queryString = """
SELECT DISTINCT ?prep ?pName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131.
    wd:Q1189 ?prep ?a .
    ?prep <http://schema.org/name> ?pName.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [4]:
#return all predicates and objects from athletes (people) 
#found P1000 'record held'

queryString = """
SELECT DISTINCT ?pred ?pName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131;
        ?pred ?obj.
    ?pred <http://schema.org/name> ?pName.
    
} 
ORDER BY DESC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P8482'), ('pName', 'Český hudební slovník osob a institucí ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P9391'), ('pName', 'Česká divadelní encyklopedie ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P2605'), ('pName', 'ČSFD person ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P8432'), ('pName', 'Österreichisches Musiklexikon Online ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P8080'), ('pName', 'Ökumenisches Heiligenlexikon ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P9415'), ('pName', 'ÖCV Biolex ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P5377'), ('pName', 'Étonnants voyageurs participant ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P9318'), ('pName', 'Ávvir topic ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P1556'), ('pName', 'zbMATH author ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P617'), ('pName', 'yard number')]
[('pred', 'http://www.wikidata.org/prop

2526

Information: 
- Query 1
* wdt:P2501 (results)
* wdt:P1000 (record held)
- Query 2

- Query 3
* wdt:P495 (country of origin)
* wdt:P27 (country of citizenship)
* wdt:P1532 (country for sport)

- Query 4
* wdt:P641 (sport) (query 4.1)
* wdt:P21 (sex or gender) (query 4.2)


Important query for the country (query 4.1 and 3)

In [41]:
#return objects connected with wdt:P1532 'country for sport' from athletes (people) 
#information for the 3rd query and 4.1 query
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1532 ?obj.
    
    ?obj <http://schema.org/name> ?objName.
} 
ORDER BY ?objName
"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q889'), ('objName', 'Afghanistan')]
[('obj', 'http://www.wikidata.org/entity/Q222'), ('objName', 'Albania')]
[('obj', 'http://www.wikidata.org/entity/Q262'), ('objName', 'Algeria')]
[('obj', 'http://www.wikidata.org/entity/Q16641'), ('objName', 'American Samoa')]
[('obj', 'http://www.wikidata.org/entity/Q228'), ('objName', 'Andorra')]
[('obj', 'http://www.wikidata.org/entity/Q168159'), ('objName', 'Anglesey')]
[('obj', 'http://www.wikidata.org/entity/Q916'), ('objName', 'Angola')]
[('obj', 'http://www.wikidata.org/entity/Q25228'), ('objName', 'Anguilla')]
[('obj', 'http://www.wikidata.org/entity/Q781'), ('objName', 'Antigua and Barbuda')]
[('obj', 'http://www.wikidata.org/entity/Q414'), ('objName', 'Argentina')]
[('obj', 'http://www.wikidata.org/entity/Q399'), ('objName', 'Armenia')]
[('obj', 'http://www.wikidata.org/entity/Q21203'), ('objName', 'Aruba')]
[('obj', 'http://www.wikidata.org/entity/Q408'), ('objName', 'Australia')]
[('obj',

335

Info: sports for each athlete (query 4.1, 4.3)

In [44]:
#return the sports of each athlete 
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P641 ?obj.
    
    ?obj <http://schema.org/name> ?objName.
} 
ORDER BY ?objName
"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q163892'), ('objName', '10,000 metres')]
[('obj', 'http://www.wikidata.org/entity/Q164761'), ('objName', '100 metres')]
[('obj', 'http://www.wikidata.org/entity/Q164731'), ('objName', '100 metres hurdles')]
[('obj', 'http://www.wikidata.org/entity/Q3500855'), ('objName', '100 yard dash')]
[('obj', 'http://www.wikidata.org/entity/Q1629556'), ('objName', '1000 metres')]
[('obj', 'http://www.wikidata.org/entity/Q170004'), ('objName', '110 metres hurdles')]
[('obj', 'http://www.wikidata.org/entity/Q191691'), ('objName', '1500 metres')]
[('obj', 'http://www.wikidata.org/entity/Q211155'), ('objName', '200 metres')]
[('obj', 'http://www.wikidata.org/entity/Q846240'), ('objName', '25 metre rapid fire pistol')]
[('obj', 'http://www.wikidata.org/entity/Q223779'), ('objName', '3000 metres')]
[('obj', 'http://www.wikidata.org/entity/Q10437559'), ('objName', '3000 metres steeplechase')]
[('obj', 'http://www.wikidata.org/entity/Q210550'), ('objName', 

528

In [49]:
#return the results of each athlete 
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P2501 ?obj.   
    ?obj <http://schema.org/name> ?objName.
} 
ORDER BY ?objName
"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q54959061'), ('objName', 'athletics result')]


1

Information about gender (query 4.2)

In [50]:
#return the gender of each athlete 
queryString = """
SELECT DISTINCT ?gender ?genderName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P21 ?gender.   
    ?gender <http://schema.org/name> ?genderName.
} 
ORDER BY ?objName
"""

print("Results")
run_query(queryString)

Results
[('gender', 'http://www.wikidata.org/entity/Q6581072'), ('genderName', 'female')]
[('gender', 'http://www.wikidata.org/entity/Q6581097'), ('genderName', 'male')]
[('gender', 'http://www.wikidata.org/entity/Q1052281'), ('genderName', 'transgender female')]
[('gender', 'http://www.wikidata.org/entity/Q2449503'), ('genderName', 'transgender male')]
[('gender', 'http://www.wikidata.org/entity/Q43445'), ('genderName', 'female organism')]
[('gender', 'http://www.wikidata.org/entity/Q1097630'), ('genderName', 'intersex')]
[('gender', 'http://www.wikidata.org/entity/Q48270'), ('genderName', 'non-binary')]
[('gender', 'http://www.wikidata.org/entity/Q27679766'), ('genderName', 'transmasculine')]
[('gender', 'http://www.wikidata.org/entity/Q12964198'), ('genderName', 'genderqueer')]
[('gender', 'http://www.wikidata.org/entity/Q44148'), ('genderName', 'male organism')]
[('gender', 'http://www.wikidata.org/entity/Q15145779'), ('genderName', 'cisgender female')]
[('gender', 'http://www.wiki

12

In [52]:
#return the record held of each athlete 
queryString = """
SELECT DISTINCT ?record ?recordName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?record.   
    ?record <http://schema.org/name> ?recordName.
} 
ORDER BY ?objName
"""

print("Results")
run_query(queryString)

Results
[('record', 'http://www.wikidata.org/entity/Q754424'), ('recordName', 'apnea')]
[('record', 'http://www.wikidata.org/entity/Q1135532'), ('recordName', 'World record progression 4 × 100 metres freestyle relay')]
[('record', 'http://www.wikidata.org/entity/Q1316274'), ('recordName', 'World record progression 1500 metres freestyle')]
[('record', 'http://www.wikidata.org/entity/Q613865'), ('recordName', 'World record progression 800 metres freestyle')]
[('record', 'http://www.wikidata.org/entity/Q668373'), ('recordName', 'World record progression 400 metres freestyle')]
[('record', 'http://www.wikidata.org/entity/Q1135718'), ('recordName', 'World record progression 4 × 200 metres freestyle relay')]
[('record', 'http://www.wikidata.org/entity/Q24255295'), ('recordName', 'junior world record')]
[('record', 'http://www.wikidata.org/entity/Q1362958'), ('recordName', 'World record progression 200 metres freestyle')]
[('record', 'http://www.wikidata.org/entity/Q9130693'), ('recordName', 

59

In [55]:
#predicates from records
queryString = """
SELECT DISTINCT ?pName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?record.
    ?record ?pred ?obj.
    ?record <http://schema.org/name> ?recordName.
    ?pred <http://schema.org/name> ?pName.
} 
ORDER BY ?pName
"""

print("Results")
run_query(queryString)

Results
[('pName', 'BNCF Thesaurus ID')]
[('pName', 'BabelNet ID')]
[('pName', 'Bibliothèque nationale de France ID')]
[('pName', 'Commons category')]
[('pName', 'Commons gallery')]
[('pName', "Enciclopèdia de l'Esport Català ID")]
[('pName', 'Encyclopædia Britannica Online ID')]
[('pName', 'Freebase ID')]
[('pName', 'French Vikidia ID')]
[('pName', 'GND ID')]
[('pName', 'Google Knowledge Graph ID')]
[('pName', 'Gran Enciclopèdia Catalana ID')]
[('pName', 'Great Russian Encyclopedia Online ID')]
[('pName', 'Human Phenotype Ontology ID')]
[('pName', 'ICD-11 (foundation)')]
[('pName', 'ICD-11 ID (MMS)')]
[('pName', 'ICD-9')]
[('pName', 'ICPC 2 ID')]
[('pName', 'IPTC NewsCode')]
[('pName', 'JSTOR topic ID')]
[('pName', 'KBpedia ID')]
[('pName', 'Latvian National Encyclopedia Online ID')]
[('pName', 'Library of Congress authority ID')]
[('pName', 'MeSH descriptor ID')]
[('pName', 'MeSH tree code')]
[('pName', 'Microsoft Academic ID')]
[('pName', 'NE.se ID')]
[('pName', 'OmegaWiki Defined M

57

In [8]:
#return the records and objects connected through 'instance of' predicate to record and athlete
queryString = """
SELECT DISTINCT ?recordName ?objName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?record.
    ?record wdt:P31 ?obj.
    ?record <http://schema.org/name> ?recordName.
    ?obj <http://schema.org/name> ?objName.
} 
ORDER BY ?pName
"""

print("Results")
run_query(queryString)

Results
[('recordName', 'Czech records in athletics'), ('objName', 'record')]
[('recordName', 'Around the world sailing record'), ('objName', 'record')]
[('recordName', '60 metres hurdles world record progression'), ('objName', 'record')]
[('recordName', 'World record progression 1500 metres freestyle'), ('objName', 'record')]
[('recordName', 'World record progression 400 metres freestyle'), ('objName', 'record')]
[('recordName', 'list of North, Central American and Caribbean records in athletics'), ('objName', 'record')]
[('recordName', 'Serbian records in athletics'), ('objName', 'record')]
[('recordName', 'Marathon world record progression'), ('objName', 'record')]
[('recordName', 'Major League Baseball single-season home run record'), ('objName', 'record')]
[('recordName', 'World record progression 4 × 200 metres freestyle relay'), ('objName', 'record')]
[('recordName', 'List of world records in athletics'), ('objName', 'record')]
[('recordName', 'Dominican records in athletics'), 

76

In [7]:
#return the objects and records connected through 'subclass of' predicate to record and athlete
queryString = """
SELECT DISTINCT ?recordName ?objName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?record.
    ?record wdt:P279 ?obj.
    ?record <http://schema.org/name> ?recordName.
    ?obj <http://schema.org/name> ?objName.
} 
ORDER BY ?pName
"""

print("Results")
run_query(queryString)

Results
[('recordName', '3000 metres steeplechase'), ('objName', 'steeplechase')]
[('recordName', 'apnea'), ('objName', 'dyspnea')]
[('recordName', 'Olympic record'), ('objName', 'record')]
[('recordName', 'apnea'), ('objName', 'abnormal breathing')]
[('recordName', '200 metres backstroke'), ('objName', 'backstroke')]
[('recordName', '100 metres'), ('objName', 'sprinting')]
[('recordName', '200 metres'), ('objName', 'sprinting')]
[('recordName', '400 metres'), ('objName', 'sprinting')]
[('recordName', '300 metres'), ('objName', 'sprinting')]
[('recordName', "Men's 100 metres world record progression"), ('objName', 'world record')]
[('recordName', 'junior world record'), ('objName', 'world record')]
[('recordName', "Women's 100 metres hurdles world record progression"), ('objName', 'world record')]
[('recordName', "Women's discus throw world record progression"), ('objName', "women's world record")]
[('recordName', "men's 10,000 meters world record in track and field"), ('objName', "men

27

In [46]:
#incoming edges in athletes
queryString = """
SELECT DISTINCT ?pName 
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131.
    ?sub ?pred ?a.
    ?pred <http://schema.org/name> ?pName.
    
} 
ORDER BY ?pName
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('pName', 'relative')]
[('pName', 'founded by')]
[('pName', 'facet of')]
[('pName', 'winner')]
[('pName', 'named after')]
[('pName', 'cast member')]
[('pName', 'producer')]
[('pName', 'creator')]
[('pName', 'different from')]
[('pName', 'father')]
[('pName', 'mother')]
[('pName', 'spouse')]
[('pName', 'head coach')]
[('pName', 'sibling')]
[('pName', 'presenter')]
[('pName', 'child')]
[('pName', 'unmarried partner')]
[('pName', 'said to be the same as')]
[('pName', 'author')]
[('pName', 'has part')]
[('pName', 'member of sports team')]
[('pName', 'participant')]
[('pName', 'composer')]
[('pName', 'executive producer')]
[('pName', 'flag bearer')]
[('pName', 'victim')]
[('pName', 'Wikidata property example')]
[('pName', 'organizer')]
[('pName', 'film editor')]
[('pName', 'performer')]
[('pName', 'significant person')]
[('pName', 'director of photography')]
[('pName', 'director')]
[('pName', 'screenwriter')]
[('pName', 'head of government')]
[('pName', 'discoverer or inventor')]
[

200

In [3]:
#check if there is a connection with World Record
#?world record - ?prep1 - ?sub - ?prep - ?a
queryString = """
SELECT DISTINCT ?prep1 ?p1Name ?sub ?sName 
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131.
    ?sub ?prep ?a.
    wd:Q688615 ?prep1 ?sub.
    ?prep <http://schema.org/name> ?pName.
    ?prep1 <http://schema.org/name> ?p1Name.
    ?sub <http://schema.org/name> ?sName.
} 
ORDER BY ?prep
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [10]:
#check if there is a connection with World Record
#?world record - ?prep1 - ?sub - ?prep - ?a
queryString = """
SELECT DISTINCT ?pred1 ?p1Name ?sub ?sName ?pred ?pName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131.
    ?sub ?pred ?a;
         ?pred1 wd:Q688615.
    ?pred <http://schema.org/name> ?pName.
    ?pred1 <http://schema.org/name> ?p1Name.
    ?sub <http://schema.org/name> ?sName.
} 
ORDER BY ?prep
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pred1', 'http://www.wikidata.org/prop/direct/P31'), ('p1Name', 'instance of'), ('sub', 'http://www.wikidata.org/entity/Q70466'), ('sName', "Men's discus throw world record progression"), ('pred', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]


1

In [60]:
#return predicates and sub ()?sub-?prep-?a)
#query already done
queryString = """
SELECT DISTINCT ?sName ?pName ?aName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131.
    ?sub ?pred ?a.
    ?pred <http://schema.org/name> ?pName.
    ?sub <http://schema.org/name> ?sName.
    ?a <http://schema.org/name> ?aName.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('sName', 'Lauren Tsai'), ('pName', 'relative'), ('aName', 'Ming Tsai')]
[('sName', 'Barefoot College'), ('pName', 'founded by'), ('aName', 'Bunker Roy')]
[('sName', 'Stefan Edberg career statistics'), ('pName', 'facet of'), ('aName', 'Stefan Edberg')]
[('sName', "1976 Men's World Open Squash Championship"), ('pName', 'winner'), ('aName', 'Geoff Hunt')]
[('sName', "tennis at the 1924 Summer Olympics – women's doubles"), ('pName', 'winner'), ('aName', 'Hazel Hotchkiss Wightman')]
[('sName', '1990 Paris Open – singles'), ('pName', 'winner'), ('aName', 'Stefan Edberg')]
[('sName', "Women's China Squash Open 2014"), ('pName', 'winner'), ('aName', 'Low Wee Wern')]
[('sName', "Men's Hong Kong squash Open 2000"), ('pName', 'winner'), ('aName', 'Peter Nicol')]
[('sName', "Men's United States Open (squash) 2001"), ('pName', 'winner'), ('aName', 'David Palmer')]
[('sName', "Men's United States Open (squash) 2000"), ('pName', 'winner'), ('aName', 'Jonathon Power')]
[('sName', "Women's Ch

100

In [9]:
# query already done
#return the objects connected to athlete through record held
#In an above query I've founded P1000(?a - 'record held)
queryString = """
SELECT DISTINCT ?aName ?pName ?oName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?obj.
    wdt:P1000 <http://schema.org/name> ?pName.
    ?obj <http://schema.org/name> ?oName.
    ?a <http://schema.org/name> ?aName.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('aName', 'Kang Chae-young'), ('pName', 'record held'), ('oName', 'list of Olympic records in archery')]
[('aName', 'Kim Je-deok'), ('pName', 'record held'), ('oName', 'list of Olympic records in archery')]
[('aName', 'An San'), ('pName', 'record held'), ('oName', 'list of Olympic records in archery')]
[('aName', 'Jang Min-hee'), ('pName', 'record held'), ('oName', 'list of Olympic records in archery')]
[('aName', 'David Hemery'), ('pName', 'record held'), ('oName', "Men's 400 metres hurdles world record progression")]
[('aName', 'Kevin Young'), ('pName', 'record held'), ('oName', "Men's 400 metres hurdles world record progression")]
[('aName', 'Glenn Davis'), ('pName', 'record held'), ('oName', "Men's 400 metres hurdles world record progression")]
[('aName', 'Johnny Gibson'), ('pName', 'record held'), ('oName', "Men's 400 metres hurdles world record progression")]
[('aName', 'Sten Pettersson'), ('pName', 'record held'), ('oName', "Men's 400 metres hurdles world record progres

100

In [62]:
#query already done
#return only distinct obj
#In an above query I've founded P1000 (?a - 'record held)
queryString = """
SELECT DISTINCT ?oName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?obj.
    wdt:P1000 <http://schema.org/name> ?pName.
    ?obj <http://schema.org/name> ?oName.
    ?a <http://schema.org/name> ?aName.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('oName', 'list of Olympic records in archery')]
[('oName', "Men's 400 metres hurdles world record progression")]
[('oName', "Women's 800 metres world record progression")]
[('oName', 'Czech records in athletics')]
[('oName', '100 metres')]
[('oName', '200 metres')]
[('oName', "Men's 100 metres world record progression")]
[('oName', 'Around the world sailing record')]
[('oName', '60 metres hurdles world record progression')]
[('oName', 'World record progression 1500 metres freestyle')]
[('oName', 'World record progression 800 metres freestyle')]
[('oName', 'World record progression 400 metres freestyle')]
[('oName', 'decathlon world record progression')]
[('oName', '10,000 metres')]
[('oName', '5000 metres')]
[('oName', "men's 10,000 meters world record in track and field")]
[('oName', "Women's heptathlon world record progression")]
[('oName', '400 metres')]
[('oName', 'pole vault')]
[('oName', '3000 metres')]
[('oName', '4 × 400 metres relay')]
[('oName', '300 metres')]
[('oN

59

In [10]:
#check the predicates outcoming from the competitions
queryString = """
SELECT DISTINCT ?pred ?predName
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions.
    ?competitions ?pred ?obj.
    
    ?pred <http://schema.org/name> ?predName.
} 
ORDER BY ?predName
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P508'), ('predName', 'BNCF Thesaurus ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P2581'), ('predName', 'BabelNet ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P268'), ('predName', 'Bibliothèque nationale de France ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P373'), ('predName', 'Commons category')]
[('pred', 'http://www.wikidata.org/prop/direct/P935'), ('predName', 'Commons gallery')]
[('pred', 'http://www.wikidata.org/prop/direct/P5513'), ('predName', "Enciclopèdia de l'Esport Català ID")]
[('pred', 'http://www.wikidata.org/prop/direct/P1417'), ('predName', 'Encyclopædia Britannica Online ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P646'), ('predName', 'Freebase ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P7818'), ('predName', 'French Vikidia ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P227'), ('predName', 'GND ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P2671'), ('predN

57

In [11]:
#check the incoming predicates to the competitions
queryString = """
SELECT DISTINCT ?pred ?predName
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions.  
    ?sub ?pred ?competitions.

    ?pred <http://schema.org/name> ?predName.
} 
ORDER BY ?predName
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P301'), ('predName', "category's main topic")]
[('pred', 'http://www.wikidata.org/prop/direct/P509'), ('predName', 'cause of death')]
[('pred', 'http://www.wikidata.org/prop/direct/P2094'), ('predName', 'competition class')]
[('pred', 'http://www.wikidata.org/prop/direct/P1889'), ('predName', 'different from')]
[('pred', 'http://www.wikidata.org/prop/direct/P425'), ('predName', 'field of this occupation')]
[('pred', 'http://www.wikidata.org/prop/direct/P101'), ('predName', 'field of work')]
[('pred', 'http://www.wikidata.org/prop/direct/P527'), ('predName', 'has part')]
[('pred', 'http://www.wikidata.org/prop/direct/P31'), ('predName', 'instance of')]
[('pred', 'http://www.wikidata.org/prop/direct/P360'), ('predName', 'is a list of')]
[('pred', 'http://www.wikidata.org/prop/direct/P1753'), ('predName', 'list related to category')]
[('pred', 'http://www.wikidata.org/prop/direct/P921'), ('predName', 'main subject')]
[('pred', 'http:/

23

In [70]:
#check the outcoming edges from world record
queryString = """
SELECT DISTINCT ?pred ?pName
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions.
    ?competitions wdt:P279 wd:Q688615.
    wd:Q688615 ?pred ?obj.
    ?pred <http://schema.org/name> ?pName.
} 
ORDER BY ?pName
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category')]
[('pred', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P8408'), ('pName', 'KBpedia ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P3417'), ('pName', 'Quora topic ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P3553'), ('pName', 'Zhihu topic ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P2910'), ('pName', 'icon')]
[('pred', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('pred', 'http://www.wikidata.org/prop/direct/P460'), ('pName', 'said to be the same as')]
[('pred', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of')]
[('pred', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category")]


10

In [12]:
#check the incoming edges from world record
queryString = """
SELECT DISTINCT ?subName ?pName
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions.
    ?competitions wdt:P279 wd:Q688615.
    ?sub ?pred wd:Q688615.
    ?pred <http://schema.org/name> ?pName.
    ?sub <http://schema.org/name> ?subName.
} 
ORDER BY ?pName
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('subName', 'Tom Lieb'), ('pName', 'award received')]
[('subName', 'Category:World records'), ('pName', "category's main topic")]
[('subName', 'world best'), ('pName', 'different from')]
[('subName', 'Singapore Airlines Flight 21'), ('pName', 'has quality')]
[('subName', 'Qantas Flights 7 and 8'), ('pName', 'has quality')]
[('subName', "Men's 400 metres hurdles world record progression"), ('pName', 'instance of')]
[('subName', 'decathlon world record progression'), ('pName', 'instance of')]
[('subName', "Women's heptathlon world record progression"), ('pName', 'instance of')]
[('subName', "Men's 4 x 400 metres relay world record progression"), ('pName', 'instance of')]
[('subName', "Men's 400 metres world record progression"), ('pName', 'instance of')]
[('subName', "men's 100 metres world record"), ('pName', 'instance of')]
[('subName', "Men's discus throw world record progression"), ('pName', 'instance of')]
[('subName', '1500 metres world record progression'), ('pName', 'ins

73

In [15]:
#check the incoming edges from world record
queryString = """
SELECT DISTINCT ?results ?resultsName ?obj
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P2501 ?results.
    ?competitions ?pred1 wd:Q688615.
    ?results ?pred ?obj.
        
    ?results <http://schema.org/name> ?resultsName.
} 
ORDER BY ?athName
"""

print("Results")
run_query(queryString)

Results
[('results', 'http://www.wikidata.org/entity/Q54959061'), ('resultsName', 'athletics result'), ('obj', 'http://wikiba.se/ontology#Item')]
[('results', 'http://www.wikidata.org/entity/Q54959061'), ('resultsName', 'athletics result'), ('obj', 'http://www.wikidata.org/entity/Q54933017')]
[('results', 'http://www.wikidata.org/entity/Q54959061'), ('resultsName', 'athletics result'), ('obj', 'athletics result')]


3

In [43]:
#return the athletes that has a record (wdt:P1000 - record held) in a competitions that is a type of world record
#[('predName', 'subclass of'), ('objName', 'world record'), ('obj', 'http://www.wikidata.org/entity/Q688615')]
queryString = """
SELECT DISTINCT ?predName ?pred ?objName
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000|^wdt:P1000 ?competitions.
    ?competitions wdt:P279|wdt:P31 wd:Q688615.    
    ?competitions ?pred ?obj.
    
    ?competitions <http://schema.org/name> ?competitionsName. 
    ?pred <http://schema.org/name> ?predName. 
    ?obj <http://schema.org/name> ?objName. 
    ?ath <http://schema.org/name> ?athName.
} 
ORDER BY DESC (?objName)
"""

print("Results")
run_query(queryString)

Results
[('predName', 'subclass of'), ('pred', 'http://www.wikidata.org/prop/direct/P279'), ('objName', 'world record')]
[('predName', 'instance of'), ('pred', 'http://www.wikidata.org/prop/direct/P31'), ('objName', 'world record')]
[('predName', 'is a list of'), ('pred', 'http://www.wikidata.org/prop/direct/P360'), ('objName', "men's world record")]
[('predName', 'competition class'), ('pred', 'http://www.wikidata.org/prop/direct/P2094'), ('objName', "men's sports")]
[('predName', 'sport'), ('pred', 'http://www.wikidata.org/prop/direct/P641'), ('objName', 'heptathlon')]
[('predName', 'instance of'), ('pred', 'http://www.wikidata.org/prop/direct/P31'), ('objName', 'criterion')]
[('predName', 'sport'), ('pred', 'http://www.wikidata.org/prop/direct/P641'), ('objName', 'athletics')]
[('predName', 'has part'), ('pred', 'http://www.wikidata.org/prop/direct/P527'), ('objName', "Women's 800 metres world record progression")]
[('predName', 'instance of'), ('pred', 'http://www.wikidata.org/prop

10

In [11]:
#return the athletes that has a record (wdt:P1000 - record held) in a competitions that is a type of world record
#there are 47 athletes
queryString = """
SELECT DISTINCT ?athName ?ath
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions.
    ?competitions wdt:P279|wdt:P31 wd:Q688615.    
    
    ?competitions <http://schema.org/name> ?competitionsName. 
    ?ath <http://schema.org/name> ?athName.
} 
ORDER BY DESC (?objName)
"""

print("Results")
run_query(queryString)

Results
[('athName', 'David Hemery'), ('ath', 'http://www.wikidata.org/entity/Q543647')]
[('athName', 'Kevin Young'), ('ath', 'http://www.wikidata.org/entity/Q313176')]
[('athName', 'Glenn Davis'), ('ath', 'http://www.wikidata.org/entity/Q549627')]
[('athName', 'Johnny Gibson'), ('ath', 'http://www.wikidata.org/entity/Q941469')]
[('athName', 'Sten Pettersson'), ('ath', 'http://www.wikidata.org/entity/Q981147')]
[('athName', 'Salvatore Morale'), ('ath', 'http://www.wikidata.org/entity/Q1343248')]
[('athName', 'Yuriy Lituyev'), ('ath', 'http://www.wikidata.org/entity/Q1338087')]
[('athName', 'Morgan Taylor'), ('ath', 'http://www.wikidata.org/entity/Q709532')]
[('athName', 'Charles Bacon'), ('ath', 'http://www.wikidata.org/entity/Q703242')]
[('athName', 'Edwin Moses'), ('ath', 'http://www.wikidata.org/entity/Q52620')]
[('athName', 'Glenn Hardin'), ('ath', 'http://www.wikidata.org/entity/Q982605')]
[('athName', 'John Akii-Bua'), ('ath', 'http://www.wikidata.org/entity/Q372259')]
[('athName

47

In [17]:
#return the athletes that has a record (wdt:P1000 - record held) in a competitions that is a type of world record
#there are no athlete
queryString = """
SELECT DISTINCT ?athName ?ath
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        ^wdt:P1000 ?competitions.
    ?competitions wdt:P279|wdt:P31 wd:Q688615.    
    
    ?competitions <http://schema.org/name> ?competitionsName. 
    ?ath <http://schema.org/name> ?athName.
} 
ORDER BY DESC (?objName)
"""

print("Results")
run_query(queryString)

Results
Empty


0

****************************** 
#### Query 1
##### Identify the BGP for detaining a world record
NOTE: For detaining a world record the athlete should beat the previous world record, so I decided to print the name of the athletes that are detaining a world record.

In [15]:
#return the competitions (and the sport) and the athletes that has a record (wdt:P1000 - record held) in a competitions that is a type 
#of world record
queryString = """
SELECT DISTINCT ?athName ?competitionsName ?sportName
WHERE { 
    ?ath wdt:P106/wdt:P279* wd:Q2066131;
        wdt:P1000 ?competitions.
    ?competitions wdt:P31 wd:Q688615;
                wdt:P641 ?sport.
    
    ?competitions <http://schema.org/name> ?competitionsName. 
    ?ath <http://schema.org/name> ?athName.
    ?sport <http://schema.org/name> ?sportName.
} 
ORDER BY DESC (?objName)
"""

print("Results")
run_query(queryString)

Results
[('athName', 'Mykola Avilov'), ('competitionsName', 'decathlon world record progression'), ('sportName', 'athletics')]
[('athName', 'Rafer Johnson'), ('competitionsName', 'decathlon world record progression'), ('sportName', 'athletics')]
[('athName', 'David Hemery'), ('competitionsName', "Men's 400 metres hurdles world record progression"), ('sportName', 'athletics')]
[('athName', 'Caitlyn Jenner'), ('competitionsName', 'decathlon world record progression'), ('sportName', 'athletics')]
[('athName', 'Daley Thompson'), ('competitionsName', 'decathlon world record progression'), ('sportName', 'athletics')]
[('athName', 'Akilles Järvinen'), ('competitionsName', 'decathlon world record progression'), ('sportName', 'athletics')]
[('athName', 'Yang Chuan-kwang'), ('competitionsName', 'decathlon world record progression'), ('sportName', 'athletics')]
[('athName', 'Tomáš Dvořák'), ('competitionsName', 'decathlon world record progression'), ('sportName', 'athletics')]
[('athName', "Dan O

55

> Initially, I've checked the objects and the subjects connected to the predicates given as input of the workflow.<br>
> This is the worst way to do the notebook, so I've decided to restart and checks all the predicates and the respective subjects and objects connected to the node '*Athlete*'.<br>
> Then, I've checked them also for the various athletes and for the first query, I've found the predicate wdt:P1000 *record held*, wdt:P641 *sport*, and wdt:P31 *country*.<br>
> Connected to the competitions using the predicates wdt:P31 *instance of*, there is the node world record.<br>
> ?ath wdt:P106/wdt:P279* wd:Q2066131, it's possible to see the reasoning at the end of the notebook in the section called 'Check after feedback'. 

 (end 1st query)
 ******************

In [ ]:
#?ath - record held (P1000) - ?competitions
#Return the relationship between competition and World record if exists (competition' outgoing edges)
#found the relations between athletes and world record
queryString = """
SELECT ?athName ?recordHeld ?world
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?worldRecord .   
    
    wdt:P1000 <http://schema.org/name> ?recordHeld.
    ?worldRecord <http://schema.org/name> ?world.
    ?ath <http://schema.org/name> ?athName.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

In [ ]:
#?ath - record held (P1000) - ?competitions
#return the predicates connected to world record 
queryString = """
SELECT DISTINCT ?pName ?pred
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?worldRecord .   
    ?worldRecord ?pred ?obj.
    
    ?pred <http://schema.org/name> ?pName.
    wdt:P1000 <http://schema.org/name> ?recordHeld.
    ?worldRecord <http://schema.org/name> ?world.
    ?ath <http://schema.org/name> ?athName.
} 
ORDER BY ?pName
LIMIT 100
"""

print("Results")
run_query(queryString)

In [2]:
#return the athlete
queryString = """
SELECT DISTINCT ?athName ?world
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?worldRecord .   
    ?worldRecord wdt:P31 wd:Q688615.
    
    wdt:P1000 <http://schema.org/name> ?recordHeld.
    ?worldRecord <http://schema.org/name> ?world.
    ?ath <http://schema.org/name> ?athName.
} 
ORDER BY ?pName
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('athName', 'Mykola Avilov'), ('world', 'decathlon world record progression')]
[('athName', 'Rafer Johnson'), ('world', 'decathlon world record progression')]
[('athName', 'David Hemery'), ('world', "Men's 400 metres hurdles world record progression")]
[('athName', 'Caitlyn Jenner'), ('world', 'decathlon world record progression')]
[('athName', 'Daley Thompson'), ('world', 'decathlon world record progression')]
[('athName', 'Akilles Järvinen'), ('world', 'decathlon world record progression')]
[('athName', 'Yang Chuan-kwang'), ('world', 'decathlon world record progression')]
[('athName', 'Tomáš Dvořák'), ('world', 'decathlon world record progression')]
[('athName', 'Jackie Joyner-Kersee'), ('world', "Women's heptathlon world record progression")]
[('athName', 'James Bausch'), ('world', 'decathlon world record progression')]
[('athName', 'Kevin Mayer'), ('world', 'decathlon world record progression')]
[('athName', 'Mel Sheppard'), ('world', "Men's 4 x 400 metres relay world reco

46

In [33]:
#outgoing edges from world record
queryString = """
SELECT DISTINCT ?predName ?pred
WHERE { 
     wd:Q688615 ?pred ?obj.
     ?obj <http://schema.org/name> ?objName.
     ?pred <http://schema.org/name> ?predName.
}
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('predName', 'instance of'), ('pred', 'http://www.wikidata.org/prop/direct/P31')]
[('predName', 'said to be the same as'), ('pred', 'http://www.wikidata.org/prop/direct/P460')]
[('predName', 'subclass of'), ('pred', 'http://www.wikidata.org/prop/direct/P279')]
[('predName', "topic's main category"), ('pred', 'http://www.wikidata.org/prop/direct/P910')]


4

In [36]:
#outgoing edges from world record
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE { 
     wd:Q688615 wdt:P910 ?obj.
     ?obj <http://schema.org/name> ?objName.
}
ORDER BY ?objName
"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q6468417'), ('objName', 'Category:World records')]


1

In [34]:
#incoming edges from world record
queryString = """
SELECT DISTINCT ?predName ?pred
WHERE { 
     ?sub ?pred wd:Q688615.
     ?pred <http://schema.org/name> ?predName.
}
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('predName', 'award received'), ('pred', 'http://www.wikidata.org/prop/direct/P166')]
[('predName', "category's main topic"), ('pred', 'http://www.wikidata.org/prop/direct/P301')]
[('predName', 'different from'), ('pred', 'http://www.wikidata.org/prop/direct/P1889')]
[('predName', 'has quality'), ('pred', 'http://www.wikidata.org/prop/direct/P1552')]
[('predName', 'instance of'), ('pred', 'http://www.wikidata.org/prop/direct/P31')]
[('predName', 'is a list of'), ('pred', 'http://www.wikidata.org/prop/direct/P360')]
[('predName', 'main subject'), ('pred', 'http://www.wikidata.org/prop/direct/P921')]
[('predName', 'record label'), ('pred', 'http://www.wikidata.org/prop/direct/P264')]
[('predName', 'said to be the same as'), ('pred', 'http://www.wikidata.org/prop/direct/P460')]
[('predName', 'significant event'), ('pred', 'http://www.wikidata.org/prop/direct/P793')]
[('predName', 'subclass of'), ('pred', 'http://www.wikidata.org/prop/direct/P279')]


11

In [37]:
#incoming edges from world record
queryString = """
SELECT DISTINCT ?subName ?sub
WHERE { 
     ?sub wdt:P301 wd:Q688615.
     ?sub <http://schema.org/name> ?subName.
}
ORDER BY ?subName
"""

print("Results")
run_query(queryString)

Results
[('subName', 'Category:World records'), ('sub', 'http://www.wikidata.org/entity/Q6468417')]


1

In [38]:
#incoming edges from world record
queryString = """
SELECT DISTINCT ?subName ?sub
WHERE { 
     ?sub wdt:P921 wd:Q688615.
     ?sub <http://schema.org/name> ?subName.
}
ORDER BY ?subName
"""

print("Results")
run_query(queryString)

Results
[('subName', 'Guinness World Records'), ('sub', 'http://www.wikidata.org/entity/Q41675')]


1

In [4]:
#check competition' predicates
queryString = """
SELECT DISTINCT ?pred ?pName
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions.
    ?competitions ?pred ?obj  .   
        
    ?pred <http://schema.org/name> ?pName.
} 
ORDER BY ?pName
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P508'), ('pName', 'BNCF Thesaurus ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P2581'), ('pName', 'BabelNet ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P268'), ('pName', 'Bibliothèque nationale de France ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category')]
[('pred', 'http://www.wikidata.org/prop/direct/P935'), ('pName', 'Commons gallery')]
[('pred', 'http://www.wikidata.org/prop/direct/P5513'), ('pName', "Enciclopèdia de l'Esport Català ID")]
[('pred', 'http://www.wikidata.org/prop/direct/P1417'), ('pName', 'Encyclopædia Britannica Online ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P7818'), ('pName', 'French Vikidia ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P227'), ('pName', 'GND ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P2671'), ('pName', 'Google Knowledge Graph ID'

57

************
### Query 2 
##### Identify the BGP for the types of competitions in which a world record is classified.


In [20]:
#return the competition and the type of competition

queryString = """
SELECT DISTINCT ?competitionName ?typeCompetitionName 
WHERE { 
    ?ath wdt:P106/wdt:P279* wd:Q2066131;
        wdt:P1000 ?competition.
    ?competition wdt:P641 ?typeCompetition;
        wdt:P31 wd:Q688615.
    
    ?typeCompetition <http://schema.org/name> ?typeCompetitionName.
    ?competition <http://schema.org/name> ?competitionName.
} 
ORDER BY ?typeCompetitionName
"""

print("Results")
run_query(queryString)

Results
[('competitionName', 'decathlon world record progression'), ('typeCompetitionName', 'athletics')]
[('competitionName', "Men's 400 metres hurdles world record progression"), ('typeCompetitionName', 'athletics')]
[('competitionName', '1500 metres world record progression'), ('typeCompetitionName', 'athletics')]
[('competitionName', "Men's 4 x 400 metres relay world record progression"), ('typeCompetitionName', 'athletics')]
[('competitionName', '800 metres world record progression'), ('typeCompetitionName', 'athletics')]
[('competitionName', '3000 metres steeplechase world record progression'), ('typeCompetitionName', 'athletics')]
[('competitionName', "Women's heptathlon world record progression"), ('typeCompetitionName', 'heptathlon')]


7

> This query is similar to the first one. <br>
> Initially, I've found the wdt:P1000 *record held*, wdt:P641 *sport*, and wdt:P31 *country*.<br>
> Then I've put all together and return only the type of competitions. <br>


(end query 2)
****************

In [31]:
#?ath - generic predicate - ?competitions -generic predicate - world record
queryString = """
SELECT ?athName ?comp ?pName ?wr
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        ?pred1 ?competitions.
    ?competitions ?pred wd:Q688615  .   
        
    wdt:P1000 <http://schema.org/name> ?recordHeld.
    wd:Q688615 <http://schema.org/name> ?wr.
    ?pred <http://schema.org/name> ?pName.
    ?pred1 <http://schema.org/name> ?p1Name.
    ?competitions <http://schema.org/name> ?comp.
    ?ath <http://schema.org/name> ?athName.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('athName', 'Usain Bolt'), ('comp', "Men's 100 metres world record progression"), ('pName', 'subclass of'), ('wr', 'world record')]
[('athName', 'Kliment Kolesnikov'), ('comp', 'junior world record'), ('pName', 'subclass of'), ('wr', 'world record')]
[('athName', 'Yaroslava Mahuchikh'), ('comp', 'junior world record'), ('pName', 'subclass of'), ('wr', 'world record')]
[('athName', 'Yordanka Donkova'), ('comp', "Women's 100 metres hurdles world record progression"), ('pName', 'subclass of'), ('wr', 'world record')]
[('athName', 'Sebastian Coe'), ('comp', '1500 metres world record progression'), ('pName', 'instance of'), ('wr', 'world record')]
[('athName', 'Sebastian Coe'), ('comp', '800 metres world record progression'), ('pName', 'instance of'), ('wr', 'world record')]
[('athName', 'David Hemery'), ('comp', "Men's 400 metres hurdles world record progression"), ('pName', 'instance of'), ('wr', 'world record')]
[('athName', 'Kevin Young'), ('comp', "Men's 400 metres hurdles wor

99

In [30]:
#?ath - generic predicate - ?competitions - P279 - world record
#added given BGP about types of world record 
queryString = """
SELECT ?athName ?p1Name ?comp ?pName ?wr
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        ?pred1 ?competitions.
    ?competitions wdt:P279 wd:Q688615  .   
        
    wdt:P1000 <http://schema.org/name> ?recordHeld.
    wd:Q688615 <http://schema.org/name> ?wr.
    wdt:P279 <http://schema.org/name> ?pName.
    ?pred1 <http://schema.org/name> ?p1Name.
    ?competitions <http://schema.org/name> ?comp.
    ?ath <http://schema.org/name> ?athName.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('athName', 'Kliment Kolesnikov'), ('p1Name', 'record held'), ('comp', 'junior world record'), ('pName', 'subclass of'), ('wr', 'world record')]
[('athName', 'Yordanka Donkova'), ('p1Name', 'record held'), ('comp', "Women's 100 metres hurdles world record progression"), ('pName', 'subclass of'), ('wr', 'world record')]
[('athName', 'Usain Bolt'), ('p1Name', 'record held'), ('comp', "Men's 100 metres world record progression"), ('pName', 'subclass of'), ('wr', 'world record')]
[('athName', 'Yaroslava Mahuchikh'), ('p1Name', 'record held'), ('comp', 'junior world record'), ('pName', 'subclass of'), ('wr', 'world record')]


4

In [13]:
#some info about world record (outcoming edges)
queryString = """
SELECT DISTINCT ?wr ?outName ?out ?objName ?obj
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        ?pred1 ?competitions.
    ?competitions wdt:P279 wd:Q688615. 
    wd:Q688615 ?out ?obj.
    
    wd:Q688615 <http://schema.org/name> ?wr.
    ?out <http://schema.org/name> ?outName.
    ?obj <http://schema.org/name> ?objName.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('wr', 'world record'), ('outName', 'subclass of'), ('out', 'http://www.wikidata.org/prop/direct/P279'), ('objName', 'record'), ('obj', 'http://www.wikidata.org/entity/Q1241356')]
[('wr', 'world record'), ('outName', 'instance of'), ('out', 'http://www.wikidata.org/prop/direct/P31'), ('objName', 'quality'), ('obj', 'http://www.wikidata.org/entity/Q1207505')]
[('wr', 'world record'), ('outName', 'instance of'), ('out', 'http://www.wikidata.org/prop/direct/P31'), ('objName', 'criterion'), ('obj', 'http://www.wikidata.org/entity/Q1789452')]
[('wr', 'world record'), ('outName', 'said to be the same as'), ('out', 'http://www.wikidata.org/prop/direct/P460'), ('objName', 'record'), ('obj', 'http://www.wikidata.org/entity/Q1241356')]
[('wr', 'world record'), ('outName', "topic's main category"), ('out', 'http://www.wikidata.org/prop/direct/P910'), ('objName', 'Category:World records'), ('obj', 'http://www.wikidata.org/entity/Q6468417')]


5

In [15]:
#founded criterion relates to world record and quality
#return at the beggining wd:Q688615 wdt:P31 ?criteria.
queryString = """
SELECT ?wr ?outName ?criteriaName ?pName ?oName
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        ?pred1 ?competitions.
    ?competitions wdt:P279 wd:Q688615. 
    wd:Q688615 ?out ?criteria.
    ?criteria ?pred ?obj.
    OPTIONAL{wd:Q688615 wdt:P31 ?criteria.}.
    
    wd:Q688615 <http://schema.org/name> ?wr.
    ?out <http://schema.org/name> ?outName.
    ?criteria <http://schema.org/name> ?criteriaName.
    ?pred <http://schema.org/name> ?pName.
    ?obj <http://schema.org/name> ?oName.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('wr', 'world record'), ('outName', 'instance of'), ('criteriaName', 'criterion'), ('pName', 'subclass of'), ('oName', 'quality')]
[('wr', 'world record'), ('outName', 'instance of'), ('criteriaName', 'criterion'), ('pName', 'subclass of'), ('oName', 'value')]
[('wr', 'world record'), ('outName', 'instance of'), ('criteriaName', 'criterion'), ('pName', 'used by'), ('oName', 'classification system')]
[('wr', 'world record'), ('outName', 'instance of'), ('criteriaName', 'criterion'), ('pName', 'subclass of'), ('oName', 'quality')]
[('wr', 'world record'), ('outName', 'instance of'), ('criteriaName', 'criterion'), ('pName', 'use'), ('oName', 'classification')]
[('wr', 'world record'), ('outName', 'instance of'), ('criteriaName', 'criterion'), ('pName', 'Wikidata property'), ('oName', 'criterion used')]
[('wr', 'world record'), ('outName', 'instance of'), ('criteriaName', 'criterion'), ('pName', 'use'), ('oName', 'classification')]
[('wr', 'world record'), ('outName', 'instance of

100

In [22]:
#founded criterion relates to world record and quality
#return only obj connected to criterion
queryString = """
SELECT DISTINCT ?criterion ?pred ?pName ?obj ?oName
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        ?pred1 ?competitions.
    ?competitions wdt:P279 wd:Q688615. 
    wd:Q688615 wdt:P31 wd:Q1789452.
    wd:Q1789452 ?pred ?obj.
    
    wd:Q1789452 <http://schema.org/name> ?criterion.
    ?pred <http://schema.org/name> ?pName.
    ?obj <http://schema.org/name> ?oName.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('criterion', 'criterion'), ('pred', 'http://www.wikidata.org/prop/direct/P1535'), ('pName', 'used by'), ('obj', 'http://www.wikidata.org/entity/Q5962346'), ('oName', 'classification system')]
[('criterion', 'criterion'), ('pred', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('obj', 'http://www.wikidata.org/entity/P1013'), ('oName', 'criterion used')]
[('criterion', 'criterion'), ('pred', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q194112'), ('oName', 'value')]
[('criterion', 'criterion'), ('pred', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q1207505'), ('oName', 'quality')]
[('criterion', 'criterion'), ('pred', 'http://www.wikidata.org/prop/direct/P366'), ('pName', 'use'), ('obj', 'http://www.wikidata.org/entity/Q13582682'), ('oName', 'classification')]


5

In [2]:
#founded criterion relates to world record and quality
#return only obj connected to criterion
queryString = """
SELECT DISTINCT ?class ?p2Name ?oName
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        ?pred1 ?competitions.
    ?competitions wdt:P279 wd:Q688615. 
    wd:Q688615 wdt:P31 wd:Q1789452.
    wd:Q1789452 ?pred wd:P1013.
    wd:P1013 ?pred2 ?obj.
   
    
    wd:Q1789452 <http://schema.org/name> ?criterion.
    ?pred2 <http://schema.org/name> ?p2Name.
    wd:P1013 <http://schema.org/name> ?class.
    ?obj <http://schema.org/name> ?oName.
} 
LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('class', 'criterion used'), ('p2Name', 'instance of'), ('oName', 'restrictive qualifier')]
[('class', 'criterion used'), ('p2Name', 'instance of'), ('oName', 'Wikidata qualifier')]
[('class', 'criterion used'), ('p2Name', 'see also'), ('oName', 'assessment')]
[('class', 'criterion used'), ('p2Name', 'see also'), ('oName', 'statement supported by')]
[('class', 'criterion used'), ('p2Name', 'see also'), ('oName', 'determination method')]


5

In [23]:
#?ath - record held (P1000) - ?competitions
#Return only the relationship between competition and World record if exists (competition' outgoing edges)
#found the relations between ath and world record
queryString = """
SELECT DISTINCT ?pName ?pred
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        ?pred wd:Q688615. 
        
    wdt:P1000 <http://schema.org/name> ?recordHeld.
    wd:Q688615 <http://schema.org/name> ?wr.
    ?pred <http://schema.org/name> ?pName.
    ?competitions <http://schema.org/name> ?comp.
    ?ath <http://schema.org/name> ?athName.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pName', 'subclass of'), ('pred', 'http://www.wikidata.org/prop/direct/P279')]
[('pName', 'instance of'), ('pred', 'http://www.wikidata.org/prop/direct/P31')]
[('pName', 'is a list of'), ('pred', 'http://www.wikidata.org/prop/direct/P360')]


3

In [33]:
#relation with ?b - P100(record held) - ?a  
queryString = """
SELECT DISTINCT ?cName ?pName ?bName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131.
    ?b wdt:P1000 ?a.
    ?c ?pred ?b.
    ?prep <http://schema.org/name> ?pName.
    ?b <http://schema.org/name> ?bName.
    ?c <http://schema.org/name> ?cName.
    ?a <http://schema.org/name> ?aName.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('cName', 'John Nada Saya'), ('pName', 'software version identifier'), ('bName', 'marathon')]
[('cName', 'John Nada Saya'), ('pName', 'tabular software version'), ('bName', 'marathon')]
[('cName', 'John Nada Saya'), ('pName', 'has edition or translation'), ('bName', 'marathon')]
[('cName', 'John Nada Saya'), ('pName', 'Fonsi Nieto'), ('bName', 'marathon')]
[('cName', 'John Nada Saya'), ('pName', "Al Stewart's albums in chronological order"), ('bName', 'marathon')]
[('cName', 'John Nada Saya'), ('pName', 'Category:Adelina Lopes Vieira'), ('bName', 'marathon')]
[('cName', 'John Nada Saya'), ('pName', 'Lubomír Stoklásek'), ('bName', 'marathon')]
[('cName', 'John Nada Saya'), ('pName', 'Ciudad Vieja'), ('bName', 'marathon')]
[('cName', 'John Nada Saya'), ('pName', 'Echániz'), ('bName', 'marathon')]
[('cName', 'John Nada Saya'), ('pName', 'Natuna'), ('bName', 'marathon')]
[('cName', 'John Nada Saya'), ('pName', "Acid Mothers Temple's albums in chronological order"), ('bName', 'mara

100

In [39]:
#return information with Usain Bolt
queryString = """
SELECT DISTINCT ?cName ?pName ?bName ?p1Name ?aName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131.
    ?b ?pred1 ?a.
    wd:Q1189 ?pred ?b.
    ?pred <http://schema.org/name> ?pName.
    ?pred1 <http://schema.org/name> ?p1Name.
    ?b <http://schema.org/name> ?bName.
    wd:Q1189 <http://schema.org/name> ?cName.
    ?a <http://schema.org/name> ?aName.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('cName', 'Usain Bolt'), ('pName', 'participant in'), ('bName', "2015 World Championships in Athletics – men's 4 × 100 metres relay"), ('p1Name', 'winner'), ('aName', 'Nesta Carter')]
[('cName', 'Usain Bolt'), ('pName', 'participant in'), ('bName', "athletics at the 2012 Summer Olympics – men's 200 metres"), ('p1Name', 'winner'), ('aName', 'Usain Bolt')]
[('cName', 'Usain Bolt'), ('pName', 'participant in'), ('bName', "athletics at the 2016 Summer Olympics – men's 4 × 100 metres relay"), ('p1Name', 'winner'), ('aName', 'Asafa Powell')]
[('cName', 'Usain Bolt'), ('pName', 'participant in'), ('bName', "athletics at the 2008 Summer Olympics – men's 200 metres"), ('p1Name', 'winner'), ('aName', 'Usain Bolt')]
[('cName', 'Usain Bolt'), ('pName', 'participant in'), ('bName', "2017 World Championships in Athletics – men's 100 metres"), ('p1Name', 'winner'), ('aName', 'Justin Gatlin')]
[('cName', 'Usain Bolt'), ('pName', 'participant in'), ('bName', "2015 World Championships in Athlet

100

In [29]:
#return all possible distinct predicate from athletes 
queryString = """
SELECT DISTINCT ?a ?aName ?test1 ?test1Name 
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131.
    OPTIONAL{?test1 wdt:P31 ?a.
        FILTER(!isLiteral(?a))
        }.
    ?a <http://schema.org/name> ?aName.
    ?test1 <http://schema.org/name> ?test1Name.
    
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q740657'), ('aName', 'Leslie Stephen'), ('test1', 'http://www.wikidata.org/entity/Q98789766'), ('test1Name', "William Godwin's Novels")]
[('a', 'http://www.wikidata.org/entity/Q438508'), ('aName', 'Verónica Cepede Royg'), ('test1', 'http://www.wikidata.org/entity/Q141425'), ('test1Name', 'Paraguay at the 2012 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q1996948'), ('aName', 'Juan David Valencia Hinestroza'), ('test1', 'http://www.wikidata.org/entity/Q5904905'), ('test1Name', 'Huerto Fierro')]
[('a', 'http://www.wikidata.org/entity/Q10558666'), ('aName', 'Housain Al-Mogahwi'), ('test1', 'http://www.wikidata.org/entity/Q6472854'), ('test1Name', 'The Contemporary Austin - Laguna Gloria')]
[('a', 'http://www.wikidata.org/entity/Q4247524'), ('aName', 'Josef Kupper'), ('test1', 'http://www.wikidata.org/entity/Q102226842'), ('test1Name', 'Josef Kupper')]


5

In [19]:
#return all possible distinct predicate from a 
queryString = """
SELECT DISTINCT ?a ?aName ?test1 ?test1Name 
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131.
    ?test1 wdt:106 ?a.
    ?a <http://schema.org/name> ?aName.
    ?test1 <http://schema.org/name> ?test1Name.
    
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [30]:
#gets the properties of all instances of athlete
#athlete- profession -  - subclasses - wd:Q2066131
#I've used the Regex if I was able to use it correctly for the next possible queries
queryString = """
SELECT DISTINCT ?aName ?pName ?bName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131.
    FILTER REGEX(?aName, "Usain Bolt")
    ?a ?prep ?b.
    ?prep <http://schema.org/name> ?pName.
    ?a <http://schema.org/name> ?aName.
    ?b <http://schema.org/name> ?bName.
} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('aName', 'Usain Bolt'), ('pName', 'record held'), ('bName', "Men's 100 metres world record progression")]
[('aName', 'Usain Bolt'), ('pName', 'record held'), ('bName', "Men's 4 × 100 metres relay world record progression")]
[('aName', 'Usain Bolt'), ('pName', 'record held'), ('bName', "men's 200 metres world record progression")]
[('aName', 'Usain Bolt'), ('pName', 'medical condition'), ('bName', 'COVID-19')]
[('aName', 'Usain Bolt'), ('pName', 'occupation'), ('bName', 'association football player')]
[('aName', 'Usain Bolt'), ('pName', 'occupation'), ('bName', 'sprinter')]
[('aName', 'Usain Bolt'), ('pName', 'described by source'), ('bName', 'Lentapedia')]
[('aName', 'Usain Bolt'), ('pName', 'described by source'), ('bName', 'Obalky knih.cz')]
[('aName', 'Usain Bolt'), ('pName', 'participant in'), ('bName', "2013 Zürich Weltklasse – men's 100 metres")]
[('aName', 'Usain Bolt'), ('pName', 'participant in'), ('bName', "2012 Lausanne Athletissima – men's 200 metres")]
[('aName',

200

In [55]:
#find the predicate from Usain Bolt
queryString = """
SELECT DISTINCT ?pName ?pred
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131.
    FILTER REGEX(?aName, "Usain Bolt")
    ?b ?pred ?a.
    ?pred <http://schema.org/name> ?pName.
    ?a <http://schema.org/name> ?aName.
    ?b <http://schema.org/name> ?bName.
} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('pName', 'winner'), ('pred', 'http://www.wikidata.org/prop/direct/P1346')]
[('pName', 'cast member'), ('pred', 'http://www.wikidata.org/prop/direct/P161')]
[('pName', "category's main topic"), ('pred', 'http://www.wikidata.org/prop/direct/P301')]
[('pName', 'participant'), ('pred', 'http://www.wikidata.org/prop/direct/P710')]
[('pName', 'main subject'), ('pred', 'http://www.wikidata.org/prop/direct/P921')]
[('pName', 'flag bearer'), ('pred', 'http://www.wikidata.org/prop/direct/P3022')]
[('pName', 'Wikidata property example'), ('pred', 'http://www.wikidata.org/prop/direct/P1855')]
[('pName', 'talk show guest'), ('pred', 'http://www.wikidata.org/prop/direct/P5030')]


8

In [60]:
#predicate winner P1346
queryString = """
SELECT DISTINCT ?bName ?b
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131.
    FILTER REGEX(?aName, "Usain Bolt")
    ?b wdt:P1346 ?a.
    ?a <http://schema.org/name> ?aName.
    ?b <http://schema.org/name> ?bName.
} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('bName', 'Laureus World Sports Award for Sportsman of the Year'), ('b', 'http://www.wikidata.org/entity/Q1808301')]
[('bName', "athletics at the 2012 Summer Olympics – men's 200 metres"), ('b', 'http://www.wikidata.org/entity/Q1649415')]
[('bName', "athletics at the 2008 Summer Olympics – men's 200 metres"), ('b', 'http://www.wikidata.org/entity/Q853003')]
[('bName', 'BBC Overseas Sports Personality of the Year'), ('b', 'http://www.wikidata.org/entity/Q3494048')]
[('bName', '2015 World Championships in Athletics'), ('b', 'http://www.wikidata.org/entity/Q1026555')]
[('bName', "2013 World Championships in Athletics – men's 200 metres"), ('b', 'http://www.wikidata.org/entity/Q14199292')]
[('bName', "2015 World Championships in Athletics – men's 200 metres"), ('b', 'http://www.wikidata.org/entity/Q20798455')]
[('bName', 'Best International Athlete ESPY Award'), ('b', 'http://www.wikidata.org/entity/Q4896696')]
[('bName', "athletics at the 2016 Summer Olympics – men's 200 metres")

15

In [62]:
#check if there is something connected to b
queryString = """
SELECT DISTINCT ?bName ?pName ?cName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131.
    FILTER REGEX(?aName, "Usain Bolt")
    ?b wdt:P1346 ?a.
    ?c ?prep ?b.
    
    ?prep <http://schema.org/name> ?pName.
    ?a <http://schema.org/name> ?aName.
    ?b <http://schema.org/name> ?bName.
    ?c <http://schema.org/name> ?cName.
} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('bName', '2015 World Championships in Athletics'), ('pName', 'participant in'), ('cName', 'Daniil Tsyplakov')]
[('bName', '2015 World Championships in Athletics'), ('pName', 'participant in'), ('cName', 'Freddy Mezones')]
[('bName', "2013 World Championships in Athletics – men's 200 metres"), ('pName', 'participant in'), ('cName', 'Adam Gemili')]
[('bName', "athletics at the 2016 Summer Olympics – men's 200 metres"), ('pName', 'participant in'), ('cName', 'Adam Gemili')]
[('bName', '2015 World Championships in Athletics'), ('pName', 'participant in'), ('cName', 'Bohdan Bondarenko')]
[('bName', '2015 World Championships in Athletics'), ('pName', 'participant in'), ('cName', 'Michael Cochrane')]
[('bName', '2015 World Championships in Athletics'), ('pName', 'participant in'), ('cName', 'Wu Qianlong')]
[('bName', '2015 World Championships in Athletics'), ('pName', 'participant in'), ('cName', 'Alina Prokopyeva')]
[('bName', '2015 World Championships in Athletics'), ('pName', 'pa

200

In [63]:
#check if there is something connected to b
queryString = """
SELECT DISTINCT ?pred ?pName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131.
    FILTER REGEX(?aName, "Usain Bolt")
    ?b wdt:P1346 ?a.
    ?c ?pred ?b.
    
    ?pred <http://schema.org/name> ?pName.
    ?a <http://schema.org/name> ?aName.
    ?b <http://schema.org/name> ?bName.
    ?c <http://schema.org/name> ?cName.
} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in')]
[('pred', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for')]
[('pred', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('pred', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('pred', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received')]
[('pred', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic")]
[('pred', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('pred', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part')]
[('pred', 'http://www.wikidata.org/prop/direct/P971'), ('pName', 'category combines topics')]


9

In [73]:
#check if there is something connected to b
#P116 award received
queryString = """
SELECT DISTINCT ?pred ?pName ?d ?dName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131.
    FILTER REGEX(?aName, "Usain Bolt")
    ?b wdt:P1346 ?a.
    ?c wdt:P166 ?b.
    ?d ?pred ?c.
    
    ?pred <http://schema.org/name> ?pName.
    ?a <http://schema.org/name> ?aName.
    ?b <http://schema.org/name> ?bName.
    ?c <http://schema.org/name> ?cName.
    ?d <http://schema.org/name> ?dName.
} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P1591'), ('pName', 'defendant'), ('d', 'http://www.wikidata.org/entity/Q41496125'), ('dName', 'trial of Lionel and Jorge Messi')]
[('pred', 'http://www.wikidata.org/prop/direct/P1363'), ('pName', 'points/goal scored by'), ('d', 'http://www.wikidata.org/entity/Q2614328'), ('dName', '2014 UEFA Champions League Final')]
[('pred', 'http://www.wikidata.org/prop/direct/P1363'), ('pName', 'points/goal scored by'), ('d', 'http://www.wikidata.org/entity/Q719830'), ('dName', '2011 UEFA Champions League Final')]
[('pred', 'http://www.wikidata.org/prop/direct/P1363'), ('pName', 'points/goal scored by'), ('d', 'http://www.wikidata.org/entity/Q1152758'), ('dName', '2011 FIFA Club World Cup Final')]
[('pred', 'http://www.wikidata.org/prop/direct/P1363'), ('pName', 'points/goal scored by'), ('d', 'http://www.wikidata.org/entity/Q629474'), ('dName', '2009 FIFA Club World Cup Final')]
[('pred', 'http://www.wikidata.org/prop/direct/P1363'), ('pName',

200

In [78]:
#check if there is something connected to b
#P116 award received
queryString = """
SELECT DISTINCT ?pred1 ?p1Name
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131.
    FILTER REGEX(?aName, "Usain Bolt")
    ?b wdt:P1346 ?a.
    ?c wdt:P166 ?b.
    ?d ?pred ?c.
    ?e ?pred1 ?d.
    
    ?pred1 <http://schema.org/name> ?p1Name.
    ?e <http://schema.org/name> ?eName.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [ ]:
#gets the properties of all instances of athlete
#athlete- profession -  - subclasses - wd:Q2066131
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131;
    ?p ?o.
    
    ?p <http://schema.org/name> ?pname

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

In [12]:
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 
    ?a wdt:P279 wd:Q2066131;
        ?p ?o.
    
    ?p <http://schema.org/name> ?pname

} 
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P10'), ('pname', 'video')]
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('pname', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('pname', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('pname', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P1149'), ('pname', 'Library of Congress Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('pname', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('pname', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('pname', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pname', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]

30

In [7]:
#properties of wd:Q2066131;

queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 
    ?a wdt:P279 wd:Q2066131;
        ?p ?o.
    FILTER(!isLiteral(?o))
    
    ?p <http://schema.org/name> ?pname

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P10'), ('pname', 'video')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('pname', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('pname', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pname', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pname', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('pname', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P176'), ('pname', 'manufacturer')]
[('p', 'http://www.wikidata.org/

46

In [10]:
#gets info about wd:Q2066131

queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131;
        ?o ?p.
    
    ?p <http://schema.org/name> ?pname

} 
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q65924683'), ('pname', 'wushu practitioner')]
[('p', 'http://www.wikidata.org/entity/Q99082843'), ('pname', 'Mansourian')]
[('p', 'http://www.wikidata.org/entity/Q845104'), ('pname', 'Huang')]
[('p', 'http://www.wikidata.org/entity/Q5'), ('pname', 'human')]
[('p', 'http://www.wikidata.org/entity/Q6581072'), ('pname', 'female')]
[('p', 'http://www.wikidata.org/entity/Q794'), ('pname', 'Iran')]
[('p', 'http://www.wikidata.org/entity/Q833'), ('pname', 'Malaysia')]
[('p', 'http://www.wikidata.org/entity/Q3479346'), ('pname', 'wushu')]
[('p', 'http://www.wikidata.org/entity/Q2718234'), ('pname', 'Semirom')]
[('p', 'http://www.wikidata.org/entity/Q180078'), ('pname', 'Kermanshah')]
[('p', 'http://www.wikidata.org/entity/Q645412'), ('pname', 'Diana')]
[('p', 'http://www.wikidata.org/entity/Q19810276'), ('pname', 'Maryam')]
[('p', 'http://www.wikidata.org/entity/Q16233681'), ('pname', 'Zulhijjah Binti Azan')]
[('p', 'http://www.wikidata.org/entity

30

In [13]:
#gets the properties of all instances of athlete
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131;
    ?p ?o. 
    FILTER(!isLiteral(?o))
    
    ?p <http://schema.org/name> ?pname

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('pname', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pname', 'sex or gender')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pname', 'country of citizenship')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('pname', 'family name')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('pname', 'given name')]
[('p', 'http://www.wikidata.org/prop/direct/P10'), ('pname', 'video')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pname', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('

100

In [16]:
#gets the properties of world record
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 
    ?a wdt:P279 wd:Q688615;
    ?p ?o. 
    
    ?p <http://schema.org/name> ?pname

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1181'), ('pname', 'numeric value')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pname', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pname', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('pname', 'is a list of')]
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('pname', 'opposite of')]
[('p', 'http://www.wikidata.org/prop/direct/P508'), ('pname', 'BNCF Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('pname', 'point in time')]
[('p', 'http://www.wikidata

19

In [18]:
#gets the properties (not literals) of world record
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 
    ?a wdt:P279 wd:Q688615;
    ?p ?o. 
    FILTER(!isLiteral(?o))
    
    ?p <http://schema.org/name> ?pname

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pname', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('pname', 'is a list of')]
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('pname', 'opposite of')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P797'), ('pname', 'authority')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category")]
[('p', 'http://www.wikidata.org/prop/direct/P953'), ('pname', 'full work available at URL')]
[('p', 'http://www.wikidata.org/prop

13

In [21]:
#? - P31 (instance of) - world record
queryString = """
SELECT DISTINCT ?p ?pname ?test1
WHERE { 
    ?a wdt:P279 wd:Q688615;
    ?p ?o. 
    OPTIONAL{
        ?a wdt:P31 ?test1
    }
    
    ?p <http://schema.org/name> ?pname

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('test1', 'http://www.wikidata.org/entity/Q13406463')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('test1', 'http://www.wikidata.org/entity/Q13406463')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('pname', 'is a list of'), ('test1', 'http://www.wikidata.org/entity/Q13406463')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('test1', 'http://www.wikidata.org/entity/Q13406463')]
[('p', 'http://www.wikidata.org/prop/direct/P1181'), ('pname', 'numeric value'), ('test1', 'http://www.wikidata.org/entity/Q1816844')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('test1', 'http://www.wikidata.org/entity/Q1816844')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('test1', 'http://www.wikidata.org/entity/Q1816844')]
[('p', 'http://www.wikidata.org/prop/direct/P6366'), ('pname', 'Micr

35

In [22]:
#? - P31 (instance of) - world record
queryString = """
SELECT DISTINCT ?test1 ?test1name
WHERE { 
    ?a wdt:P279 wd:Q688615;
    ?p ?o. 
    OPTIONAL{
        ?a wdt:P31 ?test1
    }
    
    ?test1 <http://schema.org/name> ?test1name

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('test1', 'http://www.wikidata.org/entity/Q1789452'), ('test1name', 'criterion')]
[('test1', 'http://www.wikidata.org/entity/Q1190554'), ('test1name', 'occurrence')]
[('test1', 'http://www.wikidata.org/entity/Q13406463'), ('test1name', 'Wikimedia list article')]
[('test1', 'http://www.wikidata.org/entity/Q1816844'), ('test1name', 'large number')]
[('test1', 'http://www.wikidata.org/entity/Q9356633'), ('test1name', 'maximum temperature')]


5

In [27]:
#Q1789452 - criterion
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname 
WHERE { 
    ?a wdt:P31 wd:Q1789452;
        ?p ?o.
    
    ?p <http://schema.org/name> ?pname.
    ?o <http://schema.org/name> ?oname.
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of'), ('o', 'http://www.wikidata.org/entity/Q5054566'), ('oname', 'causal inference')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of'), ('o', 'http://www.wikidata.org/entity/Q654068'), ('oname', 'visibility')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of'), ('o', 'http://www.wikidata.org/entity/Q467606'), ('oname', 'model theory')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'named after'), ('o', 'http://www.wikidata.org/entity/Q548062'), ('oname', 'Jerzy Łoś')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q23013246'), ('oname', 'copy')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q3962655'), ('oname', 'opinion')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'ht

100

In [ ]:
#?ath - record held (P1000) - ?competitions
#Return the relationship between competition and World record if exists (competition' outgoing edges)
#found the relations between athletes and world record
queryString = """
SELECT DISTINCT ?recordTypes ?recordTypesName
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131.
    ?recordTypes wdt:P279 wd:Q688615.
    wd:Q2066131 
    
    ?recordTypes <http://schema.org/name> ?recordTypesName.
   
} 
ORDER BY ?recordTypesName
"""

print("Results")
run_query(queryString)

In [20]:
#check the predicates(outcoming edges) of athlete
queryString = """
SELECT DISTINCT ?pred ?pName
WHERE { 
    ?athlete wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions;
        ?prep ?obj.
    ?competitions ?pred wd:Q688615  .   
    ?pred <http://schema.org/name> ?pName.
} 
ORDER BY ?pName
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P360'), ('pName', 'is a list of')]
[('pred', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('pred', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of')]


3

In [21]:
#check the predicates(incoming edges) of athlete
queryString = """
SELECT DISTINCT ?pred ?pName
WHERE { 
    ?athlete wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions.
    ?sub ?prep ?athlete.
    ?competitions ?pred wd:Q688615  .   
    ?pred <http://schema.org/name> ?pName.
} 
ORDER BY ?pName
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of')]
[('pred', 'http://www.wikidata.org/prop/direct/P360'), ('pName', 'is a list of')]
[('pred', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]


3

In [25]:
#check in the wd:Q2066131 nodes if there is some info about nationality (outcoming edges)
queryString = """
SELECT DISTINCT ?pred1 ?p1Name
WHERE { 
    ?athlete wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions.
    ?competitions ?pred wd:Q688615  .  
    
    wd:Q2066131 ?pred1 ?obj.
    ?pred1 <http://schema.org/name> ?p1Name.
} 
ORDER BY ?pred1
"""

print("Results")
run_query(queryString)

Results
[('pred1', 'http://www.wikidata.org/prop/direct/P1014'), ('p1Name', 'Art & Architecture Thesaurus ID')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1225'), ('p1Name', 'U.S. National Archives Identifier')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1296'), ('p1Name', 'Gran Enciclopèdia Catalana ID')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1343'), ('p1Name', 'described by source')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1424'), ('p1Name', "topic's main template")]
[('pred1', 'http://www.wikidata.org/prop/direct/P18'), ('p1Name', 'image')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1889'), ('p1Name', 'different from')]
[('pred1', 'http://www.wikidata.org/prop/direct/P227'), ('p1Name', 'GND ID')]
[('pred1', 'http://www.wikidata.org/prop/direct/P2283'), ('p1Name', 'uses')]
[('pred1', 'http://www.wikidata.org/prop/direct/P2347'), ('p1Name', 'YSO ID')]
[('pred1', 'http://www.wikidata.org/prop/direct/P244'), ('p1Name', 'Library of Congress authority

38

In [26]:
#check in the wd:Q2066131 nodes if there is some info about nationality (incoming edges)
queryString = """
SELECT DISTINCT ?pred1 ?p1Name
WHERE { 
    ?athlete wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions.
    ?competitions ?pred wd:Q688615  .  
    
    ?sub ?pred1 wd:Q2066131.
    ?pred1 <http://schema.org/name> ?p1Name.
} 
ORDER BY ?pred1
"""

print("Results")
run_query(queryString)

Results
[('pred1', 'http://www.wikidata.org/prop/direct/P101'), ('p1Name', 'field of work')]
[('pred1', 'http://www.wikidata.org/prop/direct/P106'), ('p1Name', 'occupation')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1074'), ('p1Name', 'fictional analog of')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1269'), ('p1Name', 'facet of')]
[('pred1', 'http://www.wikidata.org/prop/direct/P138'), ('p1Name', 'named after')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1423'), ('p1Name', 'template has topic')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1535'), ('p1Name', 'used by')]
[('pred1', 'http://www.wikidata.org/prop/direct/P180'), ('p1Name', 'depicts')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1889'), ('p1Name', 'different from')]
[('pred1', 'http://www.wikidata.org/prop/direct/P2360'), ('p1Name', 'intended public')]
[('pred1', 'http://www.wikidata.org/prop/direct/P279'), ('p1Name', 'subclass of')]
[('pred1', 'http://www.wikidata.org/prop/direct/P301'), ('p1N

23

In [32]:
#predicates of athletes - looking for P27 -nationality
queryString = """
SELECT DISTINCT ?athleteName ?nationality ?oName
WHERE { 
    ?athlete wdt:P106/wdt:P279 wd:Q2066131.
    ?athlete wdt:P27 ?obj.
        
    wdt:P27 <http://schema.org/name> ?nationality.
    ?athlete <http://schema.org/name> ?athleteName.
    ?obj <http://schema.org/name> ?oName.
} 

ORDER BY ?pred
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('athleteName', 'Alan Fairbairn'), ('nationality', 'country of citizenship'), ('oName', 'United Kingdom')]
[('athleteName', 'Alan Fairbairn'), ('nationality', 'country of citizenship'), ('oName', 'United Kingdom of Great Britain and Ireland')]
[('athleteName', 'Mika Monto'), ('nationality', 'country of citizenship'), ('oName', 'Finland')]
[('athleteName', 'Mir Zaman Gul'), ('nationality', 'country of citizenship'), ('oName', 'Pakistan')]
[('athleteName', 'Gonzalo Miranda'), ('nationality', 'country of citizenship'), ('oName', 'Argentina')]
[('athleteName', 'Zulhijjah Binti Azan'), ('nationality', 'country of citizenship'), ('oName', 'Malaysia')]
[('athleteName', 'Ho Ka Po'), ('nationality', 'country of citizenship'), ('oName', "People's Republic of China")]
[('athleteName', 'Ollie Holland'), ('nationality', 'country of citizenship'), ('oName', 'United Kingdom')]
[('athleteName', 'Abdulla Al-Tamimi'), ('nationality', 'country of citizenship'), ('oName', 'Qatar')]
[('athleteName

100

In [33]:
#return the nationality of athletes that are connected to world record
queryString = """
SELECT DISTINCT ?athleteName ?countryName
WHERE { 
    ?athlete wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions;
        wdt:P27 ?country.
    ?competitions ?pred wd:Q688615  .  
    
    ?athlete <http://schema.org/name> ?athleteName.
    ?country <http://schema.org/name> ?countryName.
} 
ORDER BY ?pred1
"""

print("Results")
run_query(queryString)

Results
[('athleteName', 'Cate Campbell'), ('countryName', 'Australia')]
[('athleteName', 'Mykola Avilov'), ('countryName', 'Soviet Union')]
[('athleteName', 'Rafer Johnson'), ('countryName', 'United States of America')]
[('athleteName', 'David Hemery'), ('countryName', 'United Kingdom')]
[('athleteName', 'Caitlyn Jenner'), ('countryName', 'United States of America')]
[('athleteName', 'Daley Thompson'), ('countryName', 'United Kingdom')]
[('athleteName', 'Akilles Järvinen'), ('countryName', 'Finland')]
[('athleteName', 'Yang Chuan-kwang'), ('countryName', 'Taiwan')]
[('athleteName', 'Arne Borg'), ('countryName', 'Sweden')]
[('athleteName', 'Tomáš Dvořák'), ('countryName', 'Czechoslovakia')]
[('athleteName', 'David Rudisha'), ('countryName', 'Kenya')]
[('athleteName', 'Bronte Campbell'), ('countryName', 'Australia')]
[('athleteName', 'Usain Bolt'), ('countryName', 'Jamaica')]
[('athleteName', 'George Hodgson'), ('countryName', 'Canada')]
[('athleteName', 'Jackie Joyner-Kersee'), ('count

80

In [75]:
#return the italian athletes that are connected to world record
queryString = """
SELECT DISTINCT ?athleteName ?countryName ?country
WHERE { 
    ?athlete wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions;
        wdt:P27 ?country.
    ?competitions ?pred wd:Q688615  .
    FILTER REGEX(?countryName, 'Italy').
    
    ?athlete <http://schema.org/name> ?athleteName.
    ?country <http://schema.org/name> ?countryName.
} 
ORDER BY ?pred1
"""

print("Results")
run_query(queryString)

Results
[('athleteName', 'Sara Simeoni'), ('countryName', 'Italy'), ('country', 'http://www.wikidata.org/entity/Q38')]
[('athleteName', 'Salvatore Morale'), ('countryName', 'Italy'), ('country', 'http://www.wikidata.org/entity/Q38')]


2

**********
### Query 3

##### How many world records are held by Italian athletes


In [43]:
#count the italian athletes that are connected to world record
# "italian athletes" is considered as the athletes that competed for Italy (wdt:P27) (country of citizenship)

queryString = """
SELECT COUNT(DISTINCT ?athlete)
WHERE { 
    ?athlete wdt:P106/wdt:P279* wd:Q2066131;
        wdt:P1000 ?competitions;
        wdt:P27 wd:Q38. 
    ?competitions wdt:P31 wd:Q688615  .
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '1')]


1

> I've checked the predicates connected to Usain Bolt.<br>
> Then, I've also found the predicate wdt:P1346 *winner*, but it wasn't useful.
> After these operations, I've checked again the first queries and I've found wdt:P27 *nationality*. I remember wd:Q38 *Italy* from other notebooks.

(end query 3)
******

In [31]:
#check the predicates connected to the world record node
queryString = """
SELECT DISTINCT ?predName ?pred 
WHERE { 
    wd:Q2066131 ?pred ?obj.
    ?pred <http://schema.org/name> ?predName.
} 
ORDER BY ?objName
"""

print("Results")
run_query(queryString)

Results
[('predName', 'Art & Architecture Thesaurus ID'), ('pred', 'http://www.wikidata.org/prop/direct/P1014')]
[('predName', 'U.S. National Archives Identifier'), ('pred', 'http://www.wikidata.org/prop/direct/P1225')]
[('predName', 'Gran Enciclopèdia Catalana ID'), ('pred', 'http://www.wikidata.org/prop/direct/P1296')]
[('predName', 'described by source'), ('pred', 'http://www.wikidata.org/prop/direct/P1343')]
[('predName', "topic's main template"), ('pred', 'http://www.wikidata.org/prop/direct/P1424')]
[('predName', 'image'), ('pred', 'http://www.wikidata.org/prop/direct/P18')]
[('predName', 'different from'), ('pred', 'http://www.wikidata.org/prop/direct/P1889')]
[('predName', 'GND ID'), ('pred', 'http://www.wikidata.org/prop/direct/P227')]
[('predName', 'uses'), ('pred', 'http://www.wikidata.org/prop/direct/P2283')]
[('predName', 'YSO ID'), ('pred', 'http://www.wikidata.org/prop/direct/P2347')]
[('predName', 'Library of Congress authority ID'), ('pred', 'http://www.wikidata.org/pr

38

In [33]:
#check if there is something about sport (outcoming)
queryString = """
SELECT DISTINCT ?pred1 ?pred1Name ?objName ?obj
WHERE { 
    wd:Q2066131 ?pred wd:Q349.
    wd:Q349 ?pred1 ?obj.
      
    wd:Q349 <http://schema.org/name> ?cosa.
    ?pred1 <http://schema.org/name> ?pred1Name.
    ?obj <http://schema.org/name> ?objName.
} 
ORDER BY ?pred1Name
"""

print("Results")
run_query(queryString)

Results
[('pred1', 'http://www.wikidata.org/prop/direct/P1687'), ('pred1Name', 'Wikidata property'), ('objName', 'sport'), ('obj', 'http://www.wikidata.org/entity/P641')]
[('pred1', 'http://www.wikidata.org/prop/direct/P5125'), ('pred1Name', 'Wikimedia outline'), ('objName', 'outline of sports'), ('obj', 'http://www.wikidata.org/entity/Q16000501')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1343'), ('pred1Name', 'described by source'), ('objName', 'Metropolitan Museum of Art Tagging Vocabulary'), ('obj', 'http://www.wikidata.org/entity/Q106727050')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1343'), ('pred1Name', 'described by source'), ('objName', 'Gender Glossar'), ('obj', 'http://www.wikidata.org/entity/Q106508758')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1889'), ('pred1Name', 'different from'), ('objName', 'sports discipline'), ('obj', 'http://www.wikidata.org/entity/Q2312410')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1889'), ('pred1Name', 'different 

25

In [43]:
#check if there is something about sport (outcoming)
queryString = """
SELECT DISTINCT ?objName ?obj
WHERE { 
    wd:Q2066131 ?pred wd:Q349.
    wd:Q349 ?pred1 wd:Q31629.
    wd:Q31629 ?pred2 ?obj.
  
    ?pred1 <http://schema.org/name> ?pred1Name.
    ?obj <http://schema.org/name> ?objName.
} 
ORDER BY ?objName
"""

print("Results")
run_query(queryString)

Results
[('objName', 'Category:Sports by type'), ('obj', 'http://www.wikidata.org/entity/Q7216334')]
[('objName', 'Template:Infobox sport'), ('obj', 'http://www.wikidata.org/entity/Q5901151')]
[('objName', 'class'), ('obj', 'http://www.wikidata.org/entity/Q16889133')]
[('objName', 'second-order class'), ('obj', 'http://www.wikidata.org/entity/Q24017414')]
[('objName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q349')]
[('objName', 'sport'), ('obj', 'http://www.wikidata.org/entity/P641')]
[('objName', 'sports discipline'), ('obj', 'http://www.wikidata.org/entity/Q2312410')]


7

In [48]:
#check if there is something about sport (outcoming)
queryString = """
SELECT DISTINCT ?pred2Name ?nameSportName 
WHERE { 
    wd:Q2066131 ?pred wd:Q349.
    wd:Q349 ?pred1 wd:Q31629.
    ?nameSport ?pred2 wd:Q31629. 
  
    ?pred2 <http://schema.org/name> ?pred2Name.
    ?nameSport <http://schema.org/name> ?nameSportName.
} 
ORDER BY ?nameSportName
"""

print("Results")
run_query(queryString)

Results
[('pred2Name', 'instance of'), ('nameSportName', '2000 metres')]
[('pred2Name', 'instance of'), ('nameSportName', '5 pin billiards')]
[('pred2Name', 'instance of'), ('nameSportName', '9-man')]
[('pred2Name', 'instance of'), ('nameSportName', 'Accuracy landing')]
[('pred2Name', 'instance of'), ('nameSportName', 'AcroCap')]
[('pred2Name', 'instance of'), ('nameSportName', 'Aerial dance')]
[('pred2Name', 'instance of'), ('nameSportName', 'Age of Empires II')]
[('pred2Name', 'instance of'), ('nameSportName', 'Aki Kiti')]
[('pred2Name', 'instance of'), ('nameSportName', 'Akontion')]
[('pred2Name', 'instance of'), ('nameSportName', 'American football')]
[('pred2Name', 'instance of'), ('nameSportName', 'Ampe')]
[('pred2Name', 'instance of'), ('nameSportName', 'Aquabike')]
[('pred2Name', 'instance of'), ('nameSportName', 'Aquaplaning sport')]
[('pred2Name', 'instance of'), ('nameSportName', 'Asian Games sports')]
[('pred2Name', 'instance of'), ('nameSportName', 'Australian rules footba

530

In [54]:
#return the name of the sports
queryString = """
SELECT DISTINCT ?nameSportName 
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131.
    wd:Q2066131 ?pred wd:Q349.
    wd:Q349 ?pred1 wd:Q31629.
    ?nameSport wdt:P31 wd:Q31629. 
  
    ?nameSport <http://schema.org/name> ?nameSportName.
} 
ORDER BY ?nameSportName
"""

print("Results")
run_query(queryString)

Results
[('nameSportName', '2000 metres')]
[('nameSportName', '5 pin billiards')]
[('nameSportName', '9-man')]
[('nameSportName', 'Accuracy landing')]
[('nameSportName', 'AcroCap')]
[('nameSportName', 'Aerial dance')]
[('nameSportName', 'Age of Empires II')]
[('nameSportName', 'Aki Kiti')]
[('nameSportName', 'Akontion')]
[('nameSportName', 'American football')]
[('nameSportName', 'Ampe')]
[('nameSportName', 'Aquabike')]
[('nameSportName', 'Aquaplaning sport')]
[('nameSportName', 'Asian Games sports')]
[('nameSportName', 'Australian rules football')]
[('nameSportName', 'BASE jumping')]
[('nameSportName', 'Badmini')]
[('nameSportName', 'Ballon au poing')]
[('nameSportName', 'Banger rally')]
[('nameSportName', 'Bar billiards')]
[('nameSportName', 'Basque pelota')]
[('nameSportName', 'British baseball')]
[('nameSportName', 'Buschball')]
[('nameSportName', 'Calcio Fiorentino')]
[('nameSportName', 'Carpet Bowls')]
[('nameSportName', 'Cestoball')]
[('nameSportName', 'Chilean rodeo')]
[('nameS

496

In [68]:
#return the name of the sports
queryString = """
SELECT DISTINCT ?athName ?nameSportName 
WHERE {             
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions;
        wdt:P27 ?country.
    wd:Q2066131 ?pred wd:Q349.
    wd:Q349 ?pred1 wd:Q31629.
    ?nameSport wdt:P31 wd:Q31629. 
    
    FILTER REGEX(?countryName, 'France').
  
    ?nameSport <http://schema.org/name> ?nameSportName.
    ?country <http://schema.org/name> ?countryName.
    ?ath <http://schema.org/name> ?athName.
} 
ORDER BY ?nameSportName
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [68]:
#return the name of the sports
queryString = """
SELECT DISTINCT ?athName ?nameSportName 
WHERE {             
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions;
        wdt:P27 ?country.
    wd:Q2066131 ?pred wd:Q349.
    wd:Q349 ?pred1 wd:Q31629.
    ?nameSport wdt:P31 wd:Q31629. 
    
    FILTER REGEX(?countryName, 'France').
  
    ?nameSport <http://schema.org/name> ?nameSportName.
    ?country <http://schema.org/name> ?countryName.
    ?ath <http://schema.org/name> ?athName.
} 
ORDER BY ?nameSportName
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [27]:
#check if there is something about sport (outcoming)
queryString = """
SELECT DISTINCT ?pred ?predName ?objName ?obj
WHERE { 
    wd:Q349 wdt:P1552 ?typeSport.
    ?typeSport ?pred ?obj.
    
    ?obj <http://schema.org/name> ?objName.
     ?pred <http://schema.org/name> ?predName.
} 
ORDER BY ?pred1Name
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P1424'), ('predName', "topic's main template"), ('objName', 'Template:Infobox sport'), ('obj', 'http://www.wikidata.org/entity/Q5901151')]
[('pred', 'http://www.wikidata.org/prop/direct/P1687'), ('predName', 'Wikidata property'), ('objName', 'sport'), ('obj', 'http://www.wikidata.org/entity/P641')]
[('pred', 'http://www.wikidata.org/prop/direct/P1889'), ('predName', 'different from'), ('objName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q349')]
[('pred', 'http://www.wikidata.org/prop/direct/P1889'), ('predName', 'different from'), ('objName', 'sports discipline'), ('obj', 'http://www.wikidata.org/entity/Q2312410')]
[('pred', 'http://www.wikidata.org/prop/direct/P279'), ('predName', 'subclass of'), ('objName', 'class'), ('obj', 'http://www.wikidata.org/entity/Q16889133')]
[('pred', 'http://www.wikidata.org/prop/direct/P31'), ('predName', 'instance of'), ('objName', 'second-order class'), ('obj', 'http://www.wikidata.org/ent

7

In [ ]:
#check if there is something about sport (outcoming)
queryString = """
SELECT DISTINCT ?pred1 ?pred1Name ?obj ?objName 
WHERE { 
    wd:Q2066131 ?pred wd:Q349.
    wd:Q349 ?pred1 ?obj.
    FILTER(!isLiteral(?obj)).
    ?pred1 <http://schema.org/name> ?pred1Name.
    ?obj <http://schema.org/name> ?objName.
} 
ORDER BY ?objName
"""

print("Results")
run_query(queryString)

Results
[('pred1', 'http://www.wikidata.org/prop/direct/P910'), ('pred1Name', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q1457982'), ('objName', 'Category:Sports')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1343'), ('pred1Name', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q106508758'), ('objName', 'Gender Glossar')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1343'), ('pred1Name', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q106727050'), ('objName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1151'), ('pred1Name', "topic's main Wikimedia portal"), ('obj', 'http://www.wikidata.org/entity/Q6566190'), ('objName', 'Portal:Sports')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1424'), ('pred1Name', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q14397460'), ('objName', 'Template:Infobox sport overview')]
[('pred1', 'http://www.wikidata.org/

25

In [44]:
#check the predicates connected to sport and world record
queryString = """
SELECT DISTINCT ?pred1 ?pred1Name 
WHERE { 
    wd:Q2066131 ?pred wd:Q349.
    wd:Q349 ?pred1 wd:Q688615.
    ?pred1 <http://schema.org/name> ?pred1Name.
   
} 
ORDER BY ?objName
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [45]:
#check the predicates connected to sport and world record
queryString = """
SELECT DISTINCT ?pred1 ?pred1Name 
WHERE { 
    wd:Q349 ?pred1 wd:Q688615.
    ?pred1 <http://schema.org/name> ?pred1Name.
   
} 
ORDER BY ?objName
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [21]:
#relations with sport (outgoing edges)
queryString = """
SELECT DISTINCT ?pred1 ?pred1Name 
WHERE { 
    wd:Q349 ?pred1 ?obj.
    ?pred1 <http://schema.org/name> ?pred1Name.
   
} 
ORDER BY ?objName
"""

print("Results")
run_query(queryString)

Results
[('pred1', 'http://www.wikidata.org/prop/direct/P1014'), ('pred1Name', 'Art & Architecture Thesaurus ID')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1036'), ('pred1Name', 'Dewey Decimal Classification')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1051'), ('pred1Name', 'PSH ID')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1149'), ('pred1Name', 'Library of Congress Classification')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1150'), ('pred1Name', 'Regensburg Classification')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1151'), ('pred1Name', "topic's main Wikimedia portal")]
[('pred1', 'http://www.wikidata.org/prop/direct/P1190'), ('pred1Name', 'Universal Decimal Classification')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1225'), ('pred1Name', 'U.S. National Archives Identifier')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1245'), ('pred1Name', 'OmegaWiki Defined Meaning')]
[('pred1', 'http://www.wikidata.org/prop/direct/P1282'), ('pred1N

98

In [69]:
queryString = """
SELECT ?aName ?predName ?bName
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131.
    ?b wdt:P279 wd:Q688615.
    ?a ?pred ?b.
    ?a <http://schema.org/name> ?aName.
    ?pred <http://schema.org/name> ?predName.
    ?b <http://schema.org/name> ?bName.
} 
"""

print("Results")
run_query(queryString)

Results
[('aName', 'Usain Bolt'), ('predName', 'record held'), ('bName', "Men's 100 metres world record progression")]
[('aName', 'Kliment Kolesnikov'), ('predName', 'record held'), ('bName', 'junior world record')]
[('aName', 'Yaroslava Mahuchikh'), ('predName', 'record held'), ('bName', 'junior world record')]
[('aName', 'Yordanka Donkova'), ('predName', 'record held'), ('bName', "Women's 100 metres hurdles world record progression")]


4

In [80]:
queryString = """
SELECT ?wrName ?pred1Name ?pred1 ?cName
WHERE { 
    ?athlete wdt:P106/wdt:P279 wd:Q2066131.
    ?worldrecord wdt:P279 wd:Q688615.
    ?athlete ?recordHeld ?worldrecord.
    
    ?worldrecord ?pred1 ?discipline.
    
    ?athlete <http://schema.org/name> ?aName.
    ?recordHeld <http://schema.org/name> ?predName.
    ?pred1 <http://schema.org/name> ?pred1Name.
    ?worldrecord <http://schema.org/name> ?wrName.
    ?discipline <http://schema.org/name> ?cName.
} 
"""

print("Results")
run_query(queryString)

Results
[('wrName', "Men's 100 metres world record progression"), ('pred1Name', 'sport'), ('pred1', 'http://www.wikidata.org/prop/direct/P641'), ('cName', 'athletics')]
[('wrName', "Men's 100 metres world record progression"), ('pred1Name', 'subclass of'), ('pred1', 'http://www.wikidata.org/prop/direct/P279'), ('cName', 'world record')]
[('wrName', "Men's 100 metres world record progression"), ('pred1Name', 'is a list of'), ('pred1', 'http://www.wikidata.org/prop/direct/P360'), ('cName', "men's world record")]
[('wrName', "Men's 100 metres world record progression"), ('pred1Name', 'instance of'), ('pred1', 'http://www.wikidata.org/prop/direct/P31'), ('cName', 'Wikimedia list article')]
[('wrName', 'junior world record'), ('pred1Name', 'instance of'), ('pred1', 'http://www.wikidata.org/prop/direct/P31'), ('cName', 'criterion')]
[('wrName', 'junior world record'), ('pred1Name', 'subclass of'), ('pred1', 'http://www.wikidata.org/prop/direct/P279'), ('cName', 'world record')]
[('wrName', '

10

In [92]:
#?ath - record held (P1000) - ?competitions
#Return the relationship between competition and World record if exists (competition' outgoing edges)
#found the relations between athletes and world record
queryString = """
SELECT DISTINCT ?predName ?pred ?objName ?obj
WHERE { 
    
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions.
    ?competitions ?pred wd:Q688615.   
    wd:Q2066131 ?pred1 ?obj.
        
    ?obj <http://schema.org/name> ?objName.
    ?pred1 <http://schema.org/name> ?predName.
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('predName', 'category for eponymous categories'), ('pred', 'http://www.wikidata.org/prop/direct/P360'), ('objName', 'Category:Wikipedia categories named after sportspeople'), ('obj', 'http://www.wikidata.org/entity/Q8923229')]
[('predName', 'category for eponymous categories'), ('pred', 'http://www.wikidata.org/prop/direct/P279'), ('objName', 'Category:Wikipedia categories named after sportspeople'), ('obj', 'http://www.wikidata.org/entity/Q8923229')]
[('predName', 'category for eponymous categories'), ('pred', 'http://www.wikidata.org/prop/direct/P31'), ('objName', 'Category:Wikipedia categories named after sportspeople'), ('obj', 'http://www.wikidata.org/entity/Q8923229')]
[('predName', 'described by source'), ('pred', 'http://www.wikidata.org/prop/direct/P360'), ('objName', 'Metropolitan Museum of Art Tagging Vocabulary'), ('obj', 'http://www.wikidata.org/entity/Q106727050')]
[('predName', 'described by source'), ('pred', 'http://www.wikidata.org/prop/direct/P279'), ('objN

39

In [113]:
#searching type of sport
queryString = """
SELECT DISTINCT ?pred1Name ?pred1 ?objName ?pred2Name ?pred2 ?obj1Name ?obj1
WHERE { 
    
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions;
        wdt:P27 ?country.
    ?competitions ?pred wd:Q688615.   
    wd:Q2066131 wdt:P425 ?obj.
    ?obj ?pred2 ?obj1.
    
    FILTER REGEX(?countryName, 'France').
    
    ?obj <http://schema.org/name> ?objName.
    ?obj1 <http://schema.org/name> ?obj1Name.
    ?pred2 <http://schema.org/name> ?pred2Name.
    wdt:P425 <http://schema.org/name> ?pred1Name.
    ?country <http://schema.org/name> ?countryName.
} 
ORDER BY ?pred1Name
"""

print("Results")
run_query(queryString)

Results
[('pred1Name', 'field of this occupation'), ('objName', "women's sports"), ('pred2Name', "topic's main category"), ('pred2', 'http://www.wikidata.org/prop/direct/P910'), ('obj1Name', "Category:Women's sports"), ('obj1', 'http://www.wikidata.org/entity/Q6434515')]
[('pred1Name', 'field of this occupation'), ('objName', "men's sports"), ('pred2Name', "topic's main category"), ('pred2', 'http://www.wikidata.org/prop/direct/P910'), ('obj1Name', "Category:Men's sports"), ('obj1', 'http://www.wikidata.org/entity/Q8621021')]
[('pred1Name', 'field of this occupation'), ('objName', "men's sports"), ('pred2Name', 'instance of'), ('pred2', 'http://www.wikidata.org/prop/direct/P31'), ('obj1Name', 'competition class'), ('obj1', 'http://www.wikidata.org/entity/Q22936940')]
[('pred1Name', 'field of this occupation'), ('objName', "women's sports"), ('pred2Name', 'instance of'), ('pred2', 'http://www.wikidata.org/prop/direct/P31'), ('obj1Name', 'classification in sports'), ('obj1', 'http://www.

40

In [88]:
#searching type of sport
queryString = """
SELECT DISTINCT ?typeOfSportName ?nameSportName
WHERE { 
    
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions;
        wdt:P1532 ?country.
    ?competitions ?pred wd:Q688615.   
    wd:Q2066131 wdt:P425 ?obj.
    ?obj wdt:P1552 ?typeOfSport.
    ?nameSport wdt:P31 ?typeOfSport.
    
    FILTER REGEX(?countryName, 'France').
    
    ?obj <http://schema.org/name> ?objName.
    ?typeOfSport <http://schema.org/name> ?typeOfSportName.
    wdt:P1552 <http://schema.org/name> ?pred2Name.
    wdt:P425 <http://schema.org/name> ?pred1Name.
    ?country <http://schema.org/name> ?countryName.
    ?nameSport <http://schema.org/name> ?nameSportName.
} 
ORDER BY ?nameSportName
"""

print("Results")
run_query(queryString)

Results
[('typeOfSportName', 'type of sport'), ('nameSportName', '2000 metres')]
[('typeOfSportName', 'type of sport'), ('nameSportName', '5 pin billiards')]
[('typeOfSportName', 'type of sport'), ('nameSportName', '9-man')]
[('typeOfSportName', 'type of sport'), ('nameSportName', 'Accuracy landing')]
[('typeOfSportName', 'type of sport'), ('nameSportName', 'AcroCap')]
[('typeOfSportName', 'type of sport'), ('nameSportName', 'Aerial dance')]
[('typeOfSportName', 'type of sport'), ('nameSportName', 'Age of Empires II')]
[('typeOfSportName', 'type of sport'), ('nameSportName', 'Aki Kiti')]
[('typeOfSportName', 'type of sport'), ('nameSportName', 'Akontion')]
[('typeOfSportName', 'type of sport'), ('nameSportName', 'American football')]
[('typeOfSportName', 'type of sport'), ('nameSportName', 'Ampe')]
[('typeOfSportName', 'type of sport'), ('nameSportName', 'Aquabike')]
[('typeOfSportName', 'type of sport'), ('nameSportName', 'Aquaplaning sport')]
[('typeOfSportName', 'type of sport'), ('

496

In [124]:
#return the athlete, the country, the name of the sport and the competitions
queryString = """
SELECT DISTINCT ?athleteName ?countryName ?nameSportName ?competitionsName ?wr
WHERE { 
    
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions;
        wdt:P27 ?country.
    ?competitions ?pred wd:Q688615.   
    wd:Q2066131 wdt:P425 ?obj.
    ?obj wdt:P1552 ?typeOfSport.
    ?nameSport wdt:P31 ?typeOfSport.
    
    FILTER REGEX(?countryName, 'France').
    
    ?ath <http://schema.org/name> ?athleteName.
    ?competitions <http://schema.org/name> ?competitionsName.
    wd:Q688615 <http://schema.org/name> ?wr.
    
    ?obj <http://schema.org/name> ?objName.
    ?typeOfSport <http://schema.org/name> ?typeOfSportName.
    wdt:P1552 <http://schema.org/name> ?pred2Name.
    wdt:P425 <http://schema.org/name> ?pred1Name.
    ?country <http://schema.org/name> ?countryName.
    ?nameSport <http://schema.org/name> ?nameSportName.
} 
ORDER BY ?nameSportName
"""

print("Results")
run_query(queryString)

Results
[('athleteName', 'Marie Collonvillé'), ('countryName', 'France'), ('nameSportName', '2000 metres'), ('competitionsName', 'decathlon world record progression'), ('wr', 'world record')]
[('athleteName', 'Sasha Zhoya'), ('countryName', 'France'), ('nameSportName', '2000 metres'), ('competitionsName', 'list of world junior records in athletics'), ('wr', 'world record')]
[('athleteName', 'Kevin Mayer'), ('countryName', 'France'), ('nameSportName', '2000 metres'), ('competitionsName', 'decathlon world record progression'), ('wr', 'world record')]
[('athleteName', 'Christian Plaziat'), ('countryName', 'France'), ('nameSportName', '2000 metres'), ('competitionsName', "Women's heptathlon world record progression"), ('wr', 'world record')]
[('athleteName', 'Marie Collonvillé'), ('countryName', 'France'), ('nameSportName', '5 pin billiards'), ('competitionsName', 'decathlon world record progression'), ('wr', 'world record')]
[('athleteName', 'Sasha Zhoya'), ('countryName', 'France'), ('na

1984

In [140]:
#return the athlete, the country, the name of the sport and the competitions
queryString = """
SELECT DISTINCT (COUNT(?nameSportName) AS ?count) ?nameSportName
WHERE { 
    
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions;
        wdt:P27 ?country.
    ?competitions ?pred wd:Q688615.   
    wd:Q2066131 wdt:P425 ?obj.
    ?obj wdt:P1552 ?typeOfSport.
    ?nameSport wdt:P31 ?typeOfSport.
    
    FILTER REGEX(?countryName, 'France').
    
    ?ath <http://schema.org/name> ?athleteName.
    ?competitions <http://schema.org/name> ?competitionsName.
    wd:Q688615 <http://schema.org/name> ?wr.
    
    ?obj <http://schema.org/name> ?objName.
    ?typeOfSport <http://schema.org/name> ?typeOfSportName.
    wdt:P1552 <http://schema.org/name> ?pred2Name.
    wdt:P425 <http://schema.org/name> ?pred1Name.
    ?country <http://schema.org/name> ?countryName.
    ?nameSport <http://schema.org/name> ?nameSportName.
} 
ORDER BY DESC (?count)
"""

print("Results")
run_query(queryString)

Results
[('count', '8'), ('nameSportName', 'skiing')]
[('count', '8'), ('nameSportName', 'gymnastics')]
[('count', '4'), ('nameSportName', 'hilly stage')]
[('count', '4'), ('nameSportName', 'whippet racing')]
[('count', '4'), ('nameSportName', 'bobsleigh')]
[('count', '4'), ('nameSportName', 'luge')]
[('count', '4'), ('nameSportName', 'motorcycle racing')]
[('count', '4'), ('nameSportName', 'winter pentathlon')]
[('count', '4'), ('nameSportName', 'gliding')]
[('count', '4'), ('nameSportName', 'indoor athletics')]
[('count', '4'), ('nameSportName', 'high diving')]
[('count', '4'), ('nameSportName', 'football')]
[('count', '4'), ('nameSportName', 'motorsport')]
[('count', '4'), ('nameSportName', 'baseball')]
[('count', '4'), ('nameSportName', 'Asian Games sports')]
[('count', '4'), ('nameSportName', 'trail running')]
[('count', '4'), ('nameSportName', 'roller hockey (quad)')]
[('count', '4'), ('nameSportName', 'dragon boat sport')]
[('count', '4'), ('nameSportName', 'surf ski')]
[('count

496

**************
### Query 4.1
##### In which specific sport France has the most world records

In [41]:
#the most inner subquery returns the count of each sport (maximum = 8)
#the second inner subquery returns the maximum
#then match the name of the sport with maximum number of world records

queryString = """    
SELECT ?nameSportName1 ?maxSport 
WHERE {
         ?ath1 wdt:P106/wdt:P279* wd:Q2066131;
            wdt:P1000 ?competitions1;
            wdt:P27 wd:Q142.
        ?competitions1 wdt:P31 wd:Q688615.   
        wd:Q2066131 wdt:P425 ?obj1.
        ?obj1 wdt:P1552 ?typeOfSport1.
        ?nameSport1 wdt:P31 ?typeOfSport1.

        ?nameSport1 <http://schema.org/name> ?nameSportName1.
        {
        SELECT (MAX(?countEachSport) AS ?maxSport)
        WHERE {                    
                    {    
                    SELECT (COUNT(?nameSport) AS ?countEachSport) ?nameSportName
                    WHERE { 

                        ?ath wdt:P106/wdt:P279* wd:Q2066131;
                            wdt:P1000 ?competitions;
                            wdt:P27 wd:Q142.
                        ?competitions wdt:P31 wd:Q688615.   
                        wd:Q2066131 wdt:P425 ?obj.
                        ?obj wdt:P1552 ?typeOfSport.
                        ?nameSport wdt:P31 ?typeOfSport.

                        ?nameSport <http://schema.org/name> ?nameSportName.
                    }
            }
            
            }
        }
}    
GROUP BY ?nameSportName1 ?maxSport
HAVING(COUNT(?nameSportName1) = ?maxSport)
            
"""
print("Results")
run_query(queryString)

Results
[('nameSportName1', 'skiing'), ('maxSport', '18')]
[('nameSportName1', 'gymnastics'), ('maxSport', '18')]


2

> I've checked the information about the node wd:Q249 *sport* and then I've found the node wd:Q31629 *type of sport* and finally the information about the name of sport using the predicate wdt:P31 *instance of*.<br>
> The other information is taken from the initial queries.

(end query 4.1)
***************


In [3]:
#the most inner subquery returns the count of each sport (maximum = 8)
#the second inner subquery returns the maximum
#then match the name of the sport with maximum number of world records

queryString = """    
SELECT ?nameSportName1 ?maxSport 
WHERE {
         ?ath1 wdt:P106/wdt:P279 wd:Q2066131;
            wdt:P1000 ?competitions1;
            wdt:P1532 wd:Q142;
            wdt:P641 ?nameSport1.
        ?competitions1 ?pred1 wd:Q688615.   
       

        ?nameSport1 <http://schema.org/name> ?nameSportName1.
        {
        SELECT (MAX(?countEachSport) AS ?maxSport)
        WHERE {                    
                    {    
                    SELECT (COUNT(?nameSport) AS ?countEachSport) ?nameSportName
                    WHERE { 

                        ?ath wdt:P106/wdt:P279 wd:Q2066131;
                            wdt:P1000 ?competitions;
                            wdt:P1532 wd:Q142;
                            wdt:P641 ?nameSport.
                        ?competitions ?pred wd:Q688615.   
                       

                        ?nameSport <http://schema.org/name> ?nameSportName.
                    }
            }
            
            }
        }
}    
GROUP BY ?nameSportName1 ?maxSport
HAVING(COUNT(?nameSportName1) = ?maxSport)
            
"""
print("Results")
run_query(queryString)

Results
The operation failed EndPointNotFound: it was impossible to connect with the endpoint in that address, check if it is correct. 

Response:
b'<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML//EN">\n<html>\n  <head>\n    <title>Error HTTP/1.1 404 File not found</title>\n  </head>\n  <body>\n    <h3>Error HTTP/1.1 404 File not found</h3><pre>\nThe requested URL was not found    URI  = \'/sparql\'\n  </pre></body></html>\n'


In [85]:
#query already done above 

queryString = """    
SELECT DISTINCT ?pred ?predName
WHERE {
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        ?pred ?obj.
    ?pred <http://schema.org/name> ?predName.
}    
ORDER BY ?pred     
LIMIT 500
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P10'), ('predName', 'video')]
[('pred', 'http://www.wikidata.org/prop/direct/P1000'), ('predName', 'record held')]
[('pred', 'http://www.wikidata.org/prop/direct/P1003'), ('predName', 'National Library of Romania ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P1005'), ('predName', 'Portuguese National Library ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P1006'), ('predName', 'Nationale Thesaurus voor Auteurs ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P101'), ('predName', 'field of work')]
[('pred', 'http://www.wikidata.org/prop/direct/P1014'), ('predName', 'Art & Architecture Thesaurus ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P1015'), ('predName', 'NORAF ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P1017'), ('predName', 'Vatican Library ID (former scheme)')]
[('pred', 'http://www.wikidata.org/prop/direct/P102'), ('predName', 'member of political party')]
[('pred', 'http://www.wikidata.org

500

*********
### Query 4.2
##### How many world records are held across genders

In [46]:
#wd:P21 gender or sex
#return the number of athlete divided by genders

queryString = """    
SELECT DISTINCT (COUNT(?ath) AS ?number) ?genderName
WHERE {
    ?ath wdt:P106/wdt:P279* wd:Q2066131;
        wdt:P21 ?gender;
        wdt:P1000 ?competitions.
    ?competitions wdt:P31 wd:Q688615.  
    ?gender <http://schema.org/name> ?genderName.
}    
GROUP BY (?genderName)   
LIMIT 500
"""

print("Results")
run_query(queryString)

Results
[('number', '5'), ('genderName', 'transgender female')]
[('number', '198'), ('genderName', 'male')]
[('number', '26'), ('genderName', 'female')]


3

> In the initial queries, I've found the predicate wd:P21 *gender*.

************

In [93]:
#return the sport that has the highest number of world records
queryString = """    
SELECT (COUNT(?nameSport) AS ?countEachSport) ?nameSportName
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions.
    ?competitions ?pred wd:Q688615.   
    
    wd:Q2066131 wdt:P425 ?obj.
    ?obj wdt:P1552 ?typeOfSport.
    ?nameSport wdt:P31 ?typeOfSport.

    ?nameSport <http://schema.org/name> ?nameSportName.
}  
ORDER BY DESC (?countEachSport)

"""

print("Results")
run_query(queryString)

Results
[('countEachSport', '192'), ('nameSportName', 'skiing')]
[('countEachSport', '192'), ('nameSportName', 'gymnastics')]
[('countEachSport', '96'), ('nameSportName', 'hilly stage')]
[('countEachSport', '96'), ('nameSportName', 'whippet racing')]
[('countEachSport', '96'), ('nameSportName', 'bobsleigh')]
[('countEachSport', '96'), ('nameSportName', 'luge')]
[('countEachSport', '96'), ('nameSportName', 'motorcycle racing')]
[('countEachSport', '96'), ('nameSportName', 'winter pentathlon')]
[('countEachSport', '96'), ('nameSportName', 'gliding')]
[('countEachSport', '96'), ('nameSportName', 'indoor athletics')]
[('countEachSport', '96'), ('nameSportName', 'high diving')]
[('countEachSport', '96'), ('nameSportName', 'football')]
[('countEachSport', '96'), ('nameSportName', 'motorsport')]
[('countEachSport', '96'), ('nameSportName', 'baseball')]
[('countEachSport', '96'), ('nameSportName', 'Asian Games sports')]
[('countEachSport', '96'), ('nameSportName', 'trail running')]
[('countEac

496

In [6]:
#return the sport that has the highest number of world records
queryString = """    
SELECT (COUNT(?sports) AS ?countEachSport) ?nameSportName
WHERE { 
    ?ath wdt:P106/wdt:P279 wd:Q2066131;
        wdt:P1000 ?competitions;
        wdt:P641 ?sports.
    ?competitions ?pred wd:Q688615.   

    ?sports <http://schema.org/name> ?nameSportName.
}  
ORDER BY DESC (?countEachSport)

"""

print("Results")
run_query(queryString)

Results
[('countEachSport', '60'), ('nameSportName', 'athletics')]
[('countEachSport', '36'), ('nameSportName', 'swimming')]
[('countEachSport', '7'), ('nameSportName', 'basketball')]
[('countEachSport', '2'), ('nameSportName', 'bobsleigh')]
[('countEachSport', '2'), ('nameSportName', 'association football')]
[('countEachSport', '2'), ('nameSportName', 'decathlon')]
[('countEachSport', '2'), ('nameSportName', 'water polo')]
[('countEachSport', '1'), ('nameSportName', 'auto racing')]
[('countEachSport', '1'), ('nameSportName', '400 metres hurdles')]
[('countEachSport', '1'), ('nameSportName', 'American football')]


10

***********

### Query 4.3
##### Which sport has the highest number of world records


In [21]:
#the most inner subquery returns the count of each sport (maximum = 8)
#the second inner subquery returns the maximum
#then match the name of the sport with maximum number of world records

queryString = """    
SELECT ?nameSportName1 ?maxSport 
WHERE {
         ?ath1 wdt:P106/wdt:P279 wd:Q2066131;
             wdt:P1000 ?competitions1;
             wdt:P641 ?nameSport1.
        ?competitions1 wdt:P31|wdt:P279 wd:Q688615.   


        ?nameSport1 <http://schema.org/name> ?nameSportName1.
        {
        SELECT (MAX(?countEachSport) AS ?maxSport)
        WHERE {                    
                    {    
                    SELECT (COUNT(?nameSport) AS ?countEachSport) ?nameSportName
                    WHERE { 

                        ?ath wdt:P106/wdt:P279 wd:Q2066131;
                            wdt:P1000 ?competitions;
                            wdt:P641 ?nameSport.
                        ?competitions wdt:P31|wdt:P279 wd:Q688615.   

                        ?nameSport <http://schema.org/name> ?nameSportName.
                    }
            }
            
            }
        }
}    
GROUP BY ?nameSportName1 ?maxSport
HAVING(COUNT(?nameSportName1) = ?maxSport)
            
"""

print("Results")
run_query(queryString)

Results
[('nameSportName1', 'athletics'), ('maxSport', '56')]


1

> Also in this case, I've used the information in the first queries.

*************
Check after feedback

In [8]:
#return the athletes

queryString = """
SELECT COUNT(DISTINCT ?aName)
WHERE { 

    ?a wdt:P106/wdt:P279 wd:Q2066131.
    ?a <http://schema.org/name> ?aName.
                    
} 
LIMIT 500
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '829813')]


1

In [9]:
#return the athletes

queryString = """
SELECT COUNT(DISTINCT ?aName)
WHERE { 

    ?a wdt:P106/wdt:P279* wd:Q2066131.
    ?a <http://schema.org/name> ?aName.
                    
} 
LIMIT 500
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '955403')]


1

In [13]:
#return the competitions

queryString = """
SELECT DISTINCT ?cName
WHERE { 

    ?atlhete wdt:P106/wdt:P279* wd:Q2066131.
    ?competitions wdt:P279 wd:Q688615.
    ?competitions <http://schema.org/name> ?cName.
                    
} 
ORDER BY DESC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('cName', 'largest known prime number')]
[('cName', "Men's 100 metres world record progression")]
[('cName', "women's world record")]
[('cName', 'junior world record')]
[('cName', 'highest temperature recorded on Earth')]
[('cName', "men's world record")]
[('cName', 'world best time')]
[('cName', "Women's 100 metres hurdles world record progression")]
[('cName', 'speed record')]
[('cName', 'athletics records')]
[('cName', "Men's 110 metres hurdles world record progression")]
[('cName', 'Aircraft records')]
[('cName', "Men's 20 kilometres walk world record progression")]


13

In [16]:
#return the relations between competitions and athletes

queryString = """
SELECT DISTINCT ?pName ?pred
WHERE { 

    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?competition wdt:P279 wd:Q688615.
    ?athlete ?pred ?competition.
    ?pred <http://schema.org/name> ?pName.
                    
} 
ORDER BY DESC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('pName', 'significant event'), ('pred', 'http://www.wikidata.org/prop/direct/P793')]
[('pName', 'record held'), ('pred', 'http://www.wikidata.org/prop/direct/P1000')]


2

In [4]:
#return the relations of athletes

queryString = """
SELECT DISTINCT ?pName ?pred
WHERE { 

    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?athlete ?pred ?obj.
    ?pred <http://schema.org/name> ?pName.
                    
} 
ORDER BY DESC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('pName', 'Český hudební slovník osob a institucí ID'), ('pred', 'http://www.wikidata.org/prop/direct/P8482')]
[('pName', 'Česká divadelní encyklopedie ID'), ('pred', 'http://www.wikidata.org/prop/direct/P9391')]
[('pName', 'ČSFD person ID'), ('pred', 'http://www.wikidata.org/prop/direct/P2605')]
[('pName', 'Österreichisches Musiklexikon Online ID'), ('pred', 'http://www.wikidata.org/prop/direct/P8432')]
[('pName', 'Ökumenisches Heiligenlexikon ID'), ('pred', 'http://www.wikidata.org/prop/direct/P8080')]
[('pName', 'ÖCV Biolex ID'), ('pred', 'http://www.wikidata.org/prop/direct/P9415')]
[('pName', 'Étonnants voyageurs participant ID'), ('pred', 'http://www.wikidata.org/prop/direct/P5377')]
[('pName', 'Ávvir topic ID'), ('pred', 'http://www.wikidata.org/prop/direct/P9318')]
[('pName', 'zbMATH author ID'), ('pred', 'http://www.wikidata.org/prop/direct/P1556')]
[('pName', 'yard number'), ('pred', 'http://www.wikidata.org/prop/direct/P617')]
[('pName', 'xHamster performer ID'), ('

2658

In [17]:
#return the atlhete connected to world record through record held

queryString = """
SELECT DISTINCT ?aName ?athlete
WHERE { 

    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?competition wdt:P279 wd:Q688615.
    ?athlete wdt:P1000 ?competition.
    ?athlete <http://schema.org/name> ?aName.
                    
} 
ORDER BY DESC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('aName', 'Usain Bolt'), ('athlete', 'http://www.wikidata.org/entity/Q1189')]
[('aName', 'Kliment Kolesnikov'), ('athlete', 'http://www.wikidata.org/entity/Q33259050')]
[('aName', 'Yaroslava Mahuchikh'), ('athlete', 'http://www.wikidata.org/entity/Q38236692')]
[('aName', 'Galina Zybina'), ('athlete', 'http://www.wikidata.org/entity/Q3773')]
[('aName', 'Yordanka Donkova'), ('athlete', 'http://www.wikidata.org/entity/Q233410')]


5

In [18]:
#return the predicates connected to athlete
queryString = """
SELECT DISTINCT ?pName ?pred
WHERE { 

    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?competition wdt:P279 wd:Q688615.
    ?athlete wdt:P1000 ?competition.
    ?athlete ?pred ?obj.
    ?pred <http://schema.org/name> ?pName.
                    
} 
ORDER BY DESC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('pName', 'ČSFD person ID'), ('pred', 'http://www.wikidata.org/prop/direct/P2605')]
[('pName', 'work period (start)'), ('pred', 'http://www.wikidata.org/prop/direct/P2031')]
[('pName', 'work period (end)'), ('pred', 'http://www.wikidata.org/prop/direct/P2032')]
[('pName', 'victory'), ('pred', 'http://www.wikidata.org/prop/direct/P2522')]
[('pName', "topic's main category"), ('pred', 'http://www.wikidata.org/prop/direct/P910')]
[('pName', 'spouse'), ('pred', 'http://www.wikidata.org/prop/direct/P26')]
[('pName', 'sports discipline competed in'), ('pred', 'http://www.wikidata.org/prop/direct/P2416')]
[('pName', 'sport'), ('pred', 'http://www.wikidata.org/prop/direct/P641')]
[('pName', 'social media followers'), ('pred', 'http://www.wikidata.org/prop/direct/P8687')]
[('pName', 'significant event'), ('pred', 'http://www.wikidata.org/prop/direct/P793')]
[('pName', 'short name'), ('pred', 'http://www.wikidata.org/prop/direct/P1813')]
[('pName', 'sex or gender'), ('pred', 'http://www

109

In [20]:
#return the award received
queryString = """
SELECT DISTINCT ?awardName ?award
WHERE { 

    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?competition wdt:P279 wd:Q688615.
    ?athlete wdt:P1000 ?competition.
    ?athlete wdt:P166 ?award.
    ?award <http://schema.org/name> ?awardName.
                    
} 
ORDER BY DESC (?awardName)
"""

print("Results")
run_query(queryString)

Results
[('awardName', 'Order of the Red Banner of Labour'), ('award', 'http://www.wikidata.org/entity/Q478850')]
[('awardName', 'Merited Master of Sports of the USSR'), ('award', 'http://www.wikidata.org/entity/Q9387384')]
[('awardName', 'Member of the Order of Jamaica'), ('award', 'http://www.wikidata.org/entity/Q14606267')]
[('awardName', 'Medal of the Order "For Merit to the Fatherland" I\xa0class'), ('award', 'http://www.wikidata.org/entity/Q19030539')]
[('awardName', 'Medal "For the Defence of Leningrad"'), ('award', 'http://www.wikidata.org/entity/Q924450')]
[('awardName', 'Laureus World Sports Award for Sportsman of the Year'), ('award', 'http://www.wikidata.org/entity/Q1808301')]
[('awardName', "L'Équipe Champion of Champions"), ('award', 'http://www.wikidata.org/entity/Q1061233')]
[('awardName', 'Honored Master of Sports of Russia'), ('award', 'http://www.wikidata.org/entity/Q32164')]
[('awardName', 'Commander of the Order of Distinction'), ('award', 'http://www.wikidata.org/

11

In [79]:
#return predicates connected to award and the object
queryString = """
SELECT DISTINCT ?pName ?oName
WHERE { 

    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?competition wdt:P279 wd:Q688615.
    ?athlete wdt:P1000 ?competition.
    ?athlete wdt:P166 ?award.
    ?award ?pred ?obj.
    ?pred <http://schema.org/name> ?pName.
    ?obj <http://schema.org/name> ?oName.
                    
} 
ORDER BY DESC (?oName)
"""

print("Results")
run_query(queryString)

Results
[('pName', 'instance of'), ('oName', 'sports award')]
[('pName', 'instance of'), ('oName', 'socialist order of merit')]
[('pName', 'instance of'), ('oName', 'order')]
[('pName', 'instance of'), ('oName', 'honorary sporting title')]
[('pName', 'instance of'), ('oName', 'grade of an order')]
[('pName', 'subclass of'), ('oName', 'commander')]
[('pName', 'instance of'), ('oName', 'class of award')]
[('pName', 'instance of'), ('oName', 'civil decoration')]
[('pName', 'instance of'), ('oName', 'campaign medal')]
[('pName', 'made from material'), ('oName', 'brass')]
[('pName', 'subclass of'), ('oName', 'award medal')]
[('pName', 'sport'), ('oName', 'athletics')]
[('pName', 'winner'), ('oName', 'Usain Bolt')]
[('pName', 'country'), ('oName', 'United States of America')]
[('pName', 'winner'), ('oName', 'Tiger Woods')]
[('pName', "topic's main template"), ('oName', 'Template:Order of the Red Banner of Labour')]
[('pName', 'country'), ('oName', 'Soviet Union')]
[('pName', 'location'), ('o

63

In [41]:
#try to find some relations with award and world record
queryString = """
SELECT DISTINCT ?pName ?oName ?obj1
WHERE { 

    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?competition wdt:P279 wd:Q688615.
    ?athlete wdt:P1000 ?competition.
    ?athlete wdt:P166 ?award.
    ?award ?pred ?obj.
    ?obj ?pred1 ?obj1.
    ?pred1 <http://schema.org/name> ?pName.
    ?obj1 <http://schema.org/name> ?oName.
                    
} 
ORDER BY DESC (?oName)
"""

print("Results")
run_query(queryString)

Results
[('pName', 'contains administrative territorial entity'), ('oName', 'Østfold'), ('obj1', 'http://www.wikidata.org/entity/Q50614')]
[('pName', 'contains administrative territorial entity'), ('oName', 'Île-de-France'), ('obj1', 'http://www.wikidata.org/entity/Q13917')]
[('pName', 'lowest point'), ('oName', 'Étang de Lavalduc'), ('obj1', 'http://www.wikidata.org/entity/Q3591700')]
[('pName', 'has part'), ('oName', 'zinc'), ('obj1', 'http://www.wikidata.org/entity/Q758')]
[('pName', 'instance of'), ('oName', 'yearly prize'), ('obj1', 'http://www.wikidata.org/entity/Q23719064')]
[('pName', 'occupation'), ('oName', 'writer'), ('obj1', 'http://www.wikidata.org/entity/Q36180')]
[('pName', 'copyright status as a creator'), ('oName', 'works protected by copyrights'), ('obj1', 'http://www.wikidata.org/entity/Q73555012')]
[('pName', 'position played on team / speciality'), ('oName', 'winger'), ('obj1', 'http://www.wikidata.org/entity/Q8025128')]
[('pName', 'has parts of the class'), ('oNam

2453

In [43]:
#try to find some relations with award and world record (continue iterate)
queryString = """
SELECT DISTINCT ?pName ?oName ?obj2
WHERE { 

    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?competition wdt:P279 wd:Q688615.
    ?athlete wdt:P1000 ?competition.
    ?athlete wdt:P166 ?award.
    ?award ?pred ?obj.
    ?obj ?pred1 ?obj1.
    ?obj1 ?pred2 ?obj2.
    ?pred2 <http://schema.org/name> ?pName.
    ?obj2 <http://schema.org/name> ?oName.
                    
} 
ORDER BY DESC (?oName)
LIMIT 500
"""

print("Results")
run_query(queryString)

Results
[('pName', 'uses'), ('oName', 'Ɬ'), ('obj2', 'http://www.wikidata.org/entity/Q3594624')]
[('pName', 'different from'), ('oName', '北'), ('obj2', 'http://www.wikidata.org/entity/Q51428494')]
[('pName', 'anthem'), ('oName', '’Ia ora ’o Tahiti Nui'), ('obj2', 'http://www.wikidata.org/entity/Q905475')]
[('pName', 'has edition or translation'), ('oName', 'เพลงชาติสหภาพโซเวียต (พ.ศ. 2520)'), ('obj2', 'http://www.wikidata.org/entity/Q57176992')]
[('pName', 'top-level Internet domain'), ('oName', 'ایران.'), ('obj2', 'http://www.wikidata.org/entity/Q12950808')]
[('pName', 'award received'), ('oName', 'Памятная медаль А. М. Горчакова'), ('obj2', 'http://www.wikidata.org/entity/Q4342969')]
[('pName', 'contains administrative territorial entity'), ('oName', 'ʻEua'), ('obj2', 'http://www.wikidata.org/entity/Q423528')]
[('pName', 'contains administrative territorial entity'), ('oName', 'Ștefan Vodă District'), ('obj2', 'http://www.wikidata.org/entity/Q2129061')]
[('pName', 'contains administr

500

In [53]:
#put all together
queryString = """
SELECT DISTINCT ?wrName
WHERE { 

    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?competition wdt:P279 wd:Q688615.
    ?athlete wdt:P166 ?award.
    ?award wdt:P31/wdt:P31/wdt:P279 ?worldRecord.
    ?worldRecord <http://schema.org/name> ?wrName.
                    
} 
LIMIT 500
"""

print("Results")
run_query(queryString)

Results
[('wrName', 'recognition')]
[('wrName', 'award')]
[('wrName', 'list')]
[('wrName', 'title')]
[('wrName', 'genre')]
[('wrName', 'insignia')]
[('wrName', 'name')]
[('wrName', 'role')]
[('wrName', 'Wikimedia project page')]
[('wrName', 'class')]
[('wrName', 'metaclass')]
[('wrName', 'group')]
[('wrName', 'cultural prize')]
[('wrName', 'trophy')]
[('wrName', 'rank')]
[('wrName', 'Wikimedia internal item')]
[('wrName', 'artificial entity')]
[('wrName', 'MediaWiki main-namespace page')]
[('wrName', 'type of distinction')]
[('wrName', 'fixed-order class')]
[('wrName', 'class or metaclass of Wikidata ontology')]
[('wrName', 'society')]
[('wrName', 'confraternity')]
[('wrName', 'defunct organization')]
[('wrName', 'medal')]
[('wrName', 'title of honor')]
[('wrName', 'order')]
[('wrName', 'class of award')]
[('wrName', 'gift')]
[('wrName', 'recurring event')]
[('wrName', 'legal transaction')]
[('wrName', 'group of living things')]
[('wrName', 'system')]
[('wrName', 'diagram')]
[('wrName'

104

In [83]:
#try to find some relations with award and world record (continue iterate)
#?obj ?pred1 ?obj1.
#?obj1 wdt:P279 wd:Q688615.
queryString = """
SELECT DISTINCT ?pName
WHERE { 

    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?athlete wdt:P166 ?award.
    ?athlete wdt:P1000 ?competitions.
    ?competitions wdt:P279 wd:Q688615.
    
    ?pred <http://schema.org/name> ?pName.
                    
} 
ORDER BY DESC (?oName)
LIMIT 500
"""

print("Results")
run_query(queryString)

Results
[('pName', 'record held')]


1

In [105]:
#try to find some relations with award and world record (continue iterate)
#?obj ?pred1 ?obj1.
#?obj1 wdt:P279 wd:Q688615.
queryString = """
SELECT DISTINCT ?pName ?pred
WHERE { 

    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?athlete wdt:P166 ?award.
    ?award wdt:P279 ?record.
    ?record wdt:P460 wd:Q688615.
    
    ?athlete <http://schema.org/name> ?pName.
                    
} 
ORDER BY DESC (?oName)
LIMIT 500
"""

print("Results")
run_query(queryString)

Results
[('pName', 'Tom Lieb')]


1

In [119]:
#return the athlete with world record

queryString = """
SELECT DISTINCT ?pName ?oName 
WHERE { 

    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?athlete wdt:P1000 ?competitions.
    ?competitions wdt:P279 wd:Q688615.
    
    ?athlete <http://schema.org/name> ?oName.
                    
} 
ORDER BY DESC (?oName)
LIMIT 500
"""

print("Results")
run_query(queryString)

Results
[('oName', 'Yordanka Donkova')]
[('oName', 'Yaroslava Mahuchikh')]
[('oName', 'Usain Bolt')]
[('oName', 'Kliment Kolesnikov')]
[('oName', 'Galina Zybina')]


5

In [129]:
#try to find some relations with award and world record (continue iterate)

queryString = """
SELECT DISTINCT ?pName ?obj
WHERE { 

    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?athlete wdt:P166 ?award.
    ?award ?pred ?obj.
    ?obj ?pred1 wd:Q688615.
    ?obj <http://schema.org/name> ?pName.
                    
} 
ORDER BY DESC (?pName)
LIMIT 600
"""

print("Results")
run_query(queryString)

Results
[('pName', 'record'), ('obj', 'http://www.wikidata.org/entity/Q1241356')]
[('pName', 'Category:World records'), ('obj', 'http://www.wikidata.org/entity/Q6468417')]


2

In [135]:
#try to find some relations with award and world record (continue iterate)

queryString = """
SELECT DISTINCT ?pName ?award
WHERE { 

    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?athlete wdt:P166 ?award.
    ?award wdt:P279 wd:Q1241356.
    wd:Q1241356 wdt:P460 wd:Q688615.
    ?award <http://schema.org/name> ?pName.
                    
} 
ORDER BY DESC (?pName)
LIMIT 600
"""

print("Results")
run_query(queryString)

Results
[('pName', 'world record'), ('award', 'http://www.wikidata.org/entity/Q688615')]


1

In [138]:
#try to find some relations with award and world record (continue iterate)

queryString = """
SELECT DISTINCT ?pName ?award
WHERE { 

    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?athlete wdt:P166 wd:Q688615.
    ?athlete <http://schema.org/name> ?pName.
                    
} 
ORDER BY DESC (?pName)
LIMIT 600
"""

print("Results")
run_query(queryString)

Results
[('pName', 'Tom Lieb')]


1

In [8]:
#return the predicates connected to competitions and world record
queryString = """
SELECT DISTINCT ?pName ?oName
WHERE { 
    ?ath wdt:P106/wdt:P279* wd:Q2066131;
        wdt:P1000|^wdt:P1000 ?competitions.
    ?competitions wdt:P279 wd:Q688615.
    wd:Q688615 wdt:P279 ?obj.
    ?obj ?pred ?obj1.
    ?pred <http://schema.org/name> ?pName. 
    ?obj <http://schema.org/name> ?oName. 

} 
ORDER BY DESC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('pName', "topic's main category"), ('oName', 'record')]
[('pName', 'subclass of'), ('oName', 'record')]
[('pName', 'said to be the same as'), ('oName', 'record')]
[('pName', 'has quality'), ('oName', 'record')]
[('pName', 'has effect'), ('oName', 'record')]
[('pName', 'different from'), ('oName', 'record')]
[('pName', 'described by source'), ('oName', 'record')]
[('pName', 'YSO ID'), ('oName', 'record')]
[('pName', 'Wikidata property'), ('oName', 'record')]
[('pName', 'PSH ID'), ('oName', 'record')]
[('pName', 'Klexikon article ID'), ('oName', 'record')]
[('pName', 'Gran Enciclopèdia Catalana ID'), ('oName', 'record')]
[('pName', 'Google Knowledge Graph ID'), ('oName', 'record')]
[('pName', "Enciclopèdia de l'Esport Català ID"), ('oName', 'record')]
[('pName', 'Commons category'), ('oName', 'record')]


15

In [10]:
#return the predicates connected to competitions and world record
queryString = """
SELECT COUNT(DISTINCT ?aName)
WHERE { 
    ?ath wdt:P106/wdt:P279* wd:Q2066131;
        wdt:P1000|^wdt:P1000 ?competitions.
    ?competitions wdt:P279 wd:Q688615.
    
    ?ath <http://schema.org/name> ?aName. 

} 
ORDER BY DESC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '5')]


1

In [17]:
#return the predicates connected to competitions and world record
queryString = """
SELECT COUNT(DISTINCT ?aName)
WHERE { 
    ?ath wdt:P106/wdt:P279* wd:Q2066131;
        wdt:P1000|^wdt:P1000 ?competitions.
    ?competitions wdt:P279 wd:Q688615.
    
    ?ath <http://schema.org/name> ?aName. 

} 
ORDER BY DESC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '5')]


1

In [19]:
#return the predicates connected to competitions and world record
#Marcell Jacobs hasn't a world record, so there is an error
queryString = """
SELECT DISTINCT ?aName
WHERE { 
    ?ath wdt:P106/wdt:P279* wd:Q2066131;
        wdt:P1000|^wdt:P1000 ?competitions.
    ?competitions wdt:P279* wd:Q688615.
    
    ?ath <http://schema.org/name> ?aName. 

} 
ORDER BY DESC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('aName', 'Kenenisa Bekele Beyecha')]
[('aName', 'Usain Bolt')]
[('aName', 'Renaud Lavillenie')]
[('aName', 'Rudolf Caracciola')]
[('aName', 'Sara Simeoni')]
[('aName', 'Pietro Mennea')]
[('aName', 'Marcell Jacobs')]
[('aName', 'Kliment Kolesnikov')]
[('aName', 'Taoufik Makhloufi')]
[('aName', 'Antonietta Di Martino')]
[('aName', 'Armand Duplantis')]
[('aName', 'Nina Ponomaryova')]
[('aName', 'Yaroslava Mahuchikh')]
[('aName', 'Angela Ramello')]
[('aName', 'Galina Zybina')]
[('aName', 'Yordanka Donkova')]


16

In [31]:
#try to find other athletes in award 
queryString = """
SELECT DISTINCT ?pName ?oName
WHERE { 
    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?athlete wdt:P166 ?award.
    ?award ?pred ?obj.
    ?pred <http://schema.org/name> ?pName. 
    ?obj <http://schema.org/name> ?oName. 

} 
ORDER BY DESC (?pName)
LIMIT 500
"""

print("Results")
run_query(queryString)

Results
[('pName', 'writing system'), ('oName', 'Latin script')]
[('pName', 'winner'), ('oName', 'Denzel Washington')]
[('pName', 'winner'), ('oName', 'Will Smith')]
[('pName', 'winner'), ('oName', 'Josh Hutcherson')]
[('pName', 'winner'), ('oName', 'Johnny Depp')]
[('pName', 'winner'), ('oName', 'Keanu Reeves')]
[('pName', 'winner'), ('oName', 'Jake Gyllenhaal')]
[('pName', 'winner'), ('oName', 'Eminem')]
[('pName', 'winner'), ('oName', 'Brad Pitt')]
[('pName', 'winner'), ('oName', 'Robert Pattinson')]
[('pName', 'winner'), ('oName', 'Tom Cruise')]
[('pName', 'winner'), ('oName', 'Tom Hanks')]
[('pName', 'winner'), ('oName', 'Arnold Schwarzenegger')]
[('pName', 'winner'), ('oName', 'Jim Carrey')]
[('pName', 'winner'), ('oName', 'Zac Efron')]
[('pName', 'winner'), ('oName', 'Bradley Cooper')]
[('pName', 'winner'), ('oName', 'Leonardo DiCaprio')]
[('pName', 'winner'), ('oName', 'David Hemery')]
[('pName', 'winner'), ('oName', 'Virginia Wade')]
[('pName', 'winner'), ('oName', 'David Beck

500

In [33]:
#check the subject connected to award through instance of or subclass of
queryString = """
SELECT DISTINCT ?pName ?oName
WHERE { 
    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?athlete wdt:P166 ?award.
    ?award wdt:P31|wdt:P279 ?obj.
    
    ?obj <http://schema.org/name> ?oName. 

} 
ORDER BY DESC (?oName)
LIMIT 500
"""

print("Results")
run_query(queryString)

Results
[('oName', 'zodiacal constellation')]
[('oName', 'yearly prize')]
[('oName', 'world cup')]
[('oName', 'world championship')]
[('oName', 'work with multiple executions')]
[('oName', 'work of art')]
[('oName', 'work')]
[('oName', 'word')]
[('oName', 'winter sport')]
[('oName', 'winner')]
[('oName', 'windsurf')]
[('oName', 'weapon of honor')]
[('oName', 'weapon')]
[('oName', 'watercraft class')]
[('oName', 'warrior')]
[('oName', 'walk of fame')]
[('oName', 'voluntary organization')]
[('oName', 'video game database')]
[('oName', 'video game award')]
[('oName', 'unpaid professor')]
[('oName', 'university teacher')]
[('oName', 'type of sport')]
[('oName', 'type of award')]
[('oName', 'trophy')]
[('oName', 'transport museum')]
[('oName', 'translation award')]
[('oName', 'tournament')]
[('oName', 'title of honor')]
[('oName', 'title of authority')]
[('oName', 'title')]
[('oName', 'theatre award')]
[('oName', 'termination of employment')]
[('oName', 'term')]
[('oName', 'tennis tournamen

500

In [34]:
#check the subject connected to award through instance of or subclass of
queryString = """
SELECT DISTINCT ?pName ?oName
WHERE { 
    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?athlete wdt:P166 ?award.
    ?award wdt:P31|wdt:P279 ?obj.
    ?obj ?pred ?obj1.
    
    ?pred <http://schema.org/name> ?pName.
    ?obj1 <http://schema.org/name> ?oName.

} 
ORDER BY DESC (?oName)
LIMIT 500
"""

print("Results")
run_query(queryString)

Results
[('pName', 'has part'), ('oName', 'Ōfuji Noburō Award')]
[('pName', 'different from'), ('oName', 'Óscar')]
[('pName', 'properties for this type'), ('oName', 'youth wing')]
[('pName', 'instance of'), ('oName', 'yearly prize')]
[('pName', 'different from'), ('oName', 'written work')]
[('pName', 'subclass of'), ('oName', 'written work')]
[('pName', 'sport'), ('oName', 'wrestling')]
[('pName', 'has quality'), ('oName', 'worse')]
[('pName', 'operating area'), ('oName', 'worldwide')]
[('pName', 'said to be the same as'), ('oName', 'world record')]
[('pName', 'instance of'), ('oName', 'world championship')]
[('pName', 'subclass of'), ('oName', 'world championship')]
[('pName', 'has quality'), ('oName', 'workplace conflict')]
[('pName', 'subclass of'), ('oName', 'work with multiple executions')]
[('pName', 'subclass of'), ('oName', 'work of art')]
[('pName', 'properties for this type'), ('oName', 'work location')]
[('pName', 'subclass of'), ('oName', 'work')]
[('pName', 'subject has ro

500

In [30]:
#try to find other athletes in award 
queryString = """
SELECT DISTINCT ?athlete ?aName
WHERE { 
    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?athlete wdt:P166 ?award.
    ?award wdt:P921 wd:Q688615.
    
    ?athlete <http://schema.org/name> ?aName. 

} 
ORDER BY DESC (?oName)
LIMIT 500
"""

print("Results")
run_query(queryString)

Results
[('athlete', 'http://www.wikidata.org/entity/Q24150823'), ('aName', 'Kshipra Joshi')]
[('athlete', 'http://www.wikidata.org/entity/Q3634597'), ('aName', 'Barbara Brighetti')]
[('athlete', 'http://www.wikidata.org/entity/Q11720103'), ('aName', 'Janusz Chomontek')]
[('athlete', 'http://www.wikidata.org/entity/Q29982218'), ('aName', 'Aleix Segura')]
[('athlete', 'http://www.wikidata.org/entity/Q55077729'), ('aName', 'Manvel Mamoyan')]


5

In [12]:
#return the relations of athletes

queryString = """
SELECT DISTINCT ?oName ?obj
WHERE { 

    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?athlete wdt:P1000 ?competitions.
    ?competitions ?pred ?obj.
    
    ?obj <http://schema.org/name> ?oName.
                    
} 
ORDER BY DESC (?oName)
"""

print("Results")
run_query(queryString)

Results
[('oName', 'world record'), ('obj', 'http://www.wikidata.org/entity/Q688615')]
[('oName', "women's world record"), ('obj', 'http://www.wikidata.org/entity/Q24033838')]
[('oName', "women's shot put world record progression"), ('obj', 'http://www.wikidata.org/entity/Q32161')]
[('oName', "women's high jump"), ('obj', 'http://www.wikidata.org/entity/Q55248144')]
[('oName', 'vertical jump'), ('obj', 'http://www.wikidata.org/entity/Q14596454')]
[('oName', 'type of sport'), ('obj', 'http://www.wikidata.org/entity/Q31629')]
[('oName', 'track and field'), ('obj', 'http://www.wikidata.org/entity/Q3312129')]
[('oName', 'technical disciplines of athletics'), ('obj', 'http://www.wikidata.org/entity/Q52177345')]
[('oName', 'swimming pool'), ('obj', 'http://www.wikidata.org/entity/Q1501')]
[('oName', 'swimming'), ('obj', 'http://www.wikidata.org/entity/Q31920')]
[('oName', 'swimmer'), ('obj', 'http://www.wikidata.org/entity/Q10843402')]
[('oName', 'steeplechase'), ('obj', 'http://www.wikidata

108

In [15]:
#return the relations of athletes

queryString = """
SELECT DISTINCT ?pName ?pred
WHERE { 

    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?athlete wdt:P1000 ?awards.
    ?awards ?pred wd:Q688615.
    
    ?pred <http://schema.org/name> ?pName.
                    
} 
ORDER BY DESC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('pName', 'subclass of'), ('pred', 'http://www.wikidata.org/prop/direct/P279')]
[('pName', 'is a list of'), ('pred', 'http://www.wikidata.org/prop/direct/P360')]
[('pName', 'instance of'), ('pred', 'http://www.wikidata.org/prop/direct/P31')]


3

In [21]:
#return the relations of athletes

queryString = """
SELECT DISTINCT ?aName ?athlete
WHERE { 

    ?athlete wdt:P106/wdt:P279* wd:Q2066131.
    ?athlete wdt:P1000 ?awards.
    ?awards wdt:P31/wdt:P279* wd:Q688615.
    
    ?athlete <http://schema.org/name> ?aName.
                    
} 
ORDER BY DESC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('aName', 'David Hemery'), ('athlete', 'http://www.wikidata.org/entity/Q543647')]
[('aName', 'Geoff Vanderstock'), ('athlete', 'http://www.wikidata.org/entity/Q3101062')]
[('aName', 'Kevin Young'), ('athlete', 'http://www.wikidata.org/entity/Q313176')]
[('aName', 'Glenn Davis'), ('athlete', 'http://www.wikidata.org/entity/Q549627')]
[('aName', 'Johnny Gibson'), ('athlete', 'http://www.wikidata.org/entity/Q941469')]
[('aName', 'Sten Pettersson'), ('athlete', 'http://www.wikidata.org/entity/Q981147')]
[('aName', 'Salvatore Morale'), ('athlete', 'http://www.wikidata.org/entity/Q1343248')]
[('aName', 'Charles Bacon'), ('athlete', 'http://www.wikidata.org/entity/Q703242')]
[('aName', 'Edwin Moses'), ('athlete', 'http://www.wikidata.org/entity/Q52620')]
[('aName', 'Glenn Hardin'), ('athlete', 'http://www.wikidata.org/entity/Q982605')]
[('aName', 'Rex Cawley'), ('athlete', 'http://www.wikidata.org/entity/Q1340064')]
[('aName', 'Frank Loomis'), ('athlete', 'http://www.wikidata.org/ent

48